In [ ]:
# Add the relevant scripts from LArMachineLearningData
# Nice the process so it can run with lots of cores on low priority
import os
os.nice(20)

# Add local paths
import sys
hnlDIR = os.environ['_']
sys.path.append('../pyscript')

# From pyscript Library
from Plotting import *
from Dictionary import *
from HelperFunctions import *
from CutFunctions import *

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

<h1>Specify Mass</h1>

In [ ]:
m = 260

<h1>Configuration Stuff Here</h1>

In [ ]:
ifSave = True
savePath = "../../plot_files/m"+str(m)+"_v3_selection_truly_final_loosy/"

In [ ]:
ifCut = True

In [ ]:
ifTune = False

#Common cut value
TuneCut = {
    'beamSpillBegin': 367 - 16
    , 'beamSpillEnd': 367 + 1600 + 16
    , 'crumbs': 0 
    
    , 'nHits': 50
    
    , 'MuonPeak' : 0.04
    , 'MuonEdge' : 0.04
    
    , 'ProtonPeak' : 0.96 
    , 'ProtonEdge' : 0.96 
    
    , 'PionPeak' : 0.82 
    , 'PionEdge' : 0.82 
    
    , 'ElectronPeak' : 0.96 
    , 'ElectronEdge': 0.96
    
    , 'OneShowerCenterTrackLess': 0.5
    , 'OneShowerEdgeTrackLess': 0.5
    , 'OneShowerTrackMore': 0.225
    
    , 'TwoShowerCenterTrackLess': 0.5
    , 'TwoShowerEdgeTrackLess': 0.5
    , 'TwoShowerTrackMore': 0.25    
    
    , 'OneShowerCenterFracMore': -0.12
    , 'OneShowerEdgeFracMore': -0.12
    , 'OneShowerFracLess': 0.4 
    
    , 'TwoShowerCenterFracMore': 0.0
    , 'TwoShowerEdgeFracMore': 0.0
    , 'TwoShowerFracLess': 0.4
    
    , 'OneShowerThetaCenter': 25
    , 'OneShowerThetaEdge': 25
    
    , 'TwoShowerThetaCenter': 35
    , 'TwoShowerThetaEdge': 35
    }

<h1> Read in PKL Dataframe </h1>

<h3> HNL </h3>

In [ ]:
hnl_path ="../../pkl_files/v3_April2024/df_preselect_m"+str(m)+"_v3_hnl.pkl"
print(hnl_path)
    
file_hnl = open(hnl_path, 'rb')
df_hnl = pickle.load(file_hnl)

del hnl_path
file_hnl.close()

<h3> Neutrino </h3>

In [ ]:
nu_path ="../../pkl_files/v3_April2024/df_preselect_v3_nu.pkl"
print(nu_path)
    
file_nu = open(nu_path, 'rb')
df_nu = pickle.load(file_nu)

del nu_path
file_nu.close()

<h3> Intime Cosmics </h3>

In [ ]:
cos_path ="../../pkl_files/v3_April2024/df_preselect_v3_cos.pkl"
print(cos_path)
    
file_cos = open(cos_path, 'rb')
df_cos = pickle.load(file_cos)

del cos_path
file_cos.close()

<h1>Combine FV and Non-FV HNL</h1>

In [ ]:
df_hnl['slc_true_event_type'][df_hnl['slc_true_event_type'] == 1] = 0
df_nu['slc_true_event_type'][df_nu['slc_true_event_type'] == 1] = 0
df_cos['slc_true_event_type'][df_cos['slc_true_event_type'] == 1] = 0

<h1>Scale HNL for Plotting</h1>

In [ ]:
simU = df_hnl['simU'][0]
true_counts = df_hnl['true_counts'][0]
start_counts = df_hnl['start_counts'][0]


scaleHNLPlot = 1
    
if m ==200:    
    #scaleHNLPlot = 100
    scaleHNLPlot = 10
    #scaleHNLPlot = 1
else:
    #scaleHNLPlot = 148118.81420359976 / true_counts
    scaleHNLPlot = 14811.881420359976 / true_counts

print("Sim Umu = " +str(simU))
print("true counts = " + str(true_counts))
print("start counts = " + str(start_counts))    
print("Scale factor " + str(scaleHNLPlot))

In [ ]:
plotU = getUfromScaleFactor(simU, scaleHNLPlot)

true_counts = true_counts * scaleHNLPlot
start_counts =  start_counts * scaleHNLPlot


print("Scaled Umu = " + str(plotU))
print("Scaled true counts = " + str(true_counts))
print("Scaled start counts = " + str(start_counts))

In [ ]:
df_hnl['sim_scale_pot'] = df_hnl['scale_pot']
df_hnl['scale_pot'] = df_hnl['sim_scale_pot'] * scaleHNLPlot
df_hnl['plotU'] = plotU

In [ ]:
label = str(m) + r' MeV HNL $\nu\pi^0$' + '\n' + '|U$_{{\mu 4}}$|$^{{2}}$ = ' +str(sci_notation(plotU,2,2))
print(label)

<h1> Make Beam Bucket - Post Non-clear cosmics </h1>

In [ ]:
width = 18.936

df_hnl['mod_t'] = df_hnl['slc_opt0_time_corrected_Z_pandora']%width
df_nu['mod_t'] = df_nu['slc_opt0_time_corrected_Z_pandora']%width
df_cos['mod_t'] = df_cos['slc_opt0_time_corrected_Z_pandora']%width
#----------------------------------------------
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',
                    ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_post_nonclearcosmics.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3> Save Purity and Eff </h3>

In [ ]:
purity_arr, eff_arr, cut_arr = [], [], []

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)


purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Clear Cosmics Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Beam Spill </h1>

In [ ]:
light_speed_cm_us = 29979.2458 # cm/ns

df_hnl['slc_opt0_time'] = df_hnl['slc_opt0_time_corrected_Z_pandora'] + df_hnl['slc_vtx_z']/light_speed_cm_us
df_nu['slc_opt0_time'] = df_nu['slc_opt0_time_corrected_Z_pandora'] + df_nu['slc_vtx_z']/light_speed_cm_us
df_cos['slc_opt0_time'] = df_cos['slc_opt0_time_corrected_Z_pandora'] + df_cos['slc_vtx_z']/light_speed_cm_us
#-----------------------------------#
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_opt0_time', 
                    xmin = 0, xmax = 3200, xnbin = 100,
                    xtitle = 'Opt0 Time [ns]',
                    ifAddLegend = True, addLegend = label
                    )

plt.axvline(x = TuneCut['beamSpillBegin'], c = 'r', linestyle = '--', lw = 1.5)
plt.axvline(x = TuneCut['beamSpillEnd'], c = 'r', linestyle = '--', lw = 1.5)

plt.ylim(0, 1.25e6)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beamspill.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3>Apply Cut</h3>

In [ ]:
if ifCut:
    df_hnl = cutBeamGate(df_hnl, gateBegin = TuneCut['beamSpillBegin'], gateEnd = TuneCut['beamSpillEnd'])
    df_nu = cutBeamGate(df_nu, gateBegin = TuneCut['beamSpillBegin'], gateEnd = TuneCut['beamSpillEnd'])
    df_cos = cutBeamGate(df_cos, gateBegin = TuneCut['beamSpillBegin'], gateEnd = TuneCut['beamSpillEnd'])

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Beam Spill")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Make Beam Bucket - Post Beam Spill</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',
                    ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_post_beamspill.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1> CRUMBS </h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_crumbs_score', 
                    xmin = -2, xmax =1, xnbin = 30,
                    xtitle = 'CRUMBS Score',
                    ifAddLegend = True, addLegend = label
                    )

plt.axvline(x = TuneCut['crumbs'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(0, 4.5e6), xytext=(0.4, 4.5e6)
             , arrowprops=dict(arrowstyle= '<|-', color='r', lw=1.5, ls='--'))
plt.ylim(0, 5e6)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"crumbs_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3>Apply Cut</h3>

In [ ]:
score = TuneCut['crumbs']

print(score)

if ifCut:
    df_hnl = cutCosmics(df_hnl, crumbsScore = score)
    df_nu = cutCosmics(df_nu, crumbsScore = score)
    df_cos = cutCosmics(df_cos, crumbsScore = score)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("CRUMBS")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Make Beam Bucket - Post Crumbs</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + " [ns]",
                    ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_post_crumbs.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1> Fiducial Volume </h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_is_fv', 
                    xmin = -0.5, xmax = 2.5, xnbin = 40,
                    xtitle = 'Vertex is in Fiducial Volume',
                     ifAddLegend = True, addLegend = label
                    )

plt.xticks(range(0, 2), labels=["False", "True"])

plt.axvline(x = 0.5, c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(0.5, 1.6e7), xytext=(0.9, 1.6e7)
             , arrowprops=dict(arrowstyle= '<|-',color='r',lw=1.5,ls='--'))
plt.ylim(0, 1.8e7)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"fv_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3>Apply Cut</h3>

In [ ]:
if ifCut:
    df_hnl = cutFV(df_hnl)
    df_nu = cutFV(df_nu)
    df_cos = cutFV(df_cos)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Vertex In FV")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Make Beam Bucket - Post FV</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + " [ns]",
                     ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_post_FV.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1> Defined Peak Center vs Edge</h1>

In [ ]:
lb = 4
ub = 15
#-----------------------------------#
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + " [ns]",
                     ifAddLegend = True, addLegend = label
                    )

plt.axvline(x =  lb, c = 'r', linestyle = '--', lw = 1.5)
plt.axvline(x =  ub, c = 'r', linestyle = '--', lw = 1.5)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_post_cosmic_rejection_peak_boundary.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1> N Hits </h1>

In [ ]:
df_hnl_prim = get_primary_shw_df(df_hnl)

df_nu_prim = get_primary_shw_df(df_nu)

df_cos_prim = get_primary_shw_df(df_cos)

In [ ]:
when_hnl_center = (df_hnl_prim['mod_t'] >= lb) &  (df_hnl_prim['mod_t'] <= ub)
when_nu_center = (df_nu_prim['mod_t'] >= lb) &  (df_nu_prim['mod_t'] <= ub)
when_cos_center = (df_cos_prim['mod_t'] >= lb) &  (df_cos_prim['mod_t'] <= ub)

when_hnl_edge = (df_hnl_prim['mod_t'] < lb) | (df_hnl_prim['mod_t'] > ub)
when_nu_edge = (df_nu_prim['mod_t'] < lb) | (df_nu_prim['mod_t'] > ub)
when_cos_edge = (df_cos_prim['mod_t'] < lb) | (df_cos_prim['mod_t'] > ub)

df_hnl_prim_center = df_hnl_prim[when_hnl_center]
df_nu_prim_center = df_nu_prim[when_nu_center]
df_cos_prim_center = df_cos_prim[when_cos_center]

df_hnl_prim_edge = df_hnl_prim[when_hnl_edge]
df_nu_prim_edge = df_nu_prim[when_nu_edge]
df_cos_prim_edge = df_cos_prim[when_cos_edge]

In [ ]:
plot_slc_var(df_hnl_prim, df_nu_prim, df_cos_prim,
                    true_counts, start_counts, 
                    'slc_pfp_n_hits', 
                    xmin = 0, xmax = 3000, xnbin = 60,
                    xtitle = 'n Hits',
                  ifAddLegend = True, addLegend = label
                    )

plt.annotate("", xy=(50, 2.9e5), xytext=(300, 2.9e5)
             , arrowprops=dict(arrowstyle= '<|-',color='r',lw=1.5,ls='--'))
plt.axvline(x =  TuneCut['nHits'], c = 'r', linestyle = '--', lw = 1.5)
plt.ylim(0, 3e5)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nHits.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
plot_slc_var(df_hnl_prim_center, df_nu_prim_center, df_cos_prim_center,
                    true_counts, start_counts, 
                    'slc_pfp_n_hits', 
                    xmin = 0, xmax = 2000, xnbin = 40,
                    xtitle = 'n Hits',
                  ifAddLegend = True, addLegend = label
                    )

plt.annotate("", xy=(50, 2.9e5), xytext=(300, 2.9e5)
             , arrowprops=dict(arrowstyle= '<|-',color='r',lw=1.5,ls='--'))
plt.axvline(x =  TuneCut['nHits'], c = 'r', linestyle = '--', lw = 1.5)
plt.ylim(0, 3e5)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nHits_peak.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
plot_slc_var(df_hnl_prim_edge, df_nu_prim_edge, df_cos_prim_edge,
                    true_counts, start_counts, 
                    'slc_pfp_n_hits', 
                    xmin = 0, xmax = 2000, xnbin = 40,
                    xtitle = 'n Hits',
                  ifAddLegend = True, addLegend = label
                    )
plt.annotate("", xy=(50, 1.4e4), xytext=(300, 1.4e4)
             , arrowprops=dict(arrowstyle= '<|-',color='r',lw=1.5,ls='--'))
plt.axvline(x =  TuneCut['nHits'], c = 'r', linestyle = '--', lw = 1.5)
plt.ylim(0, 1.5e4)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nHits_edge.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3>Apply Cut</h3>

In [ ]:
nHits = TuneCut['nHits']
if ifCut:
    df_hnl_prim_center = df_hnl_prim_center[df_hnl_prim_center['slc_pfp_n_hits'] >= nHits]
    df_nu_prim_center = df_nu_prim_center[df_nu_prim_center['slc_pfp_n_hits'] >= nHits]
    df_cos_prim_center = df_cos_prim_center[df_cos_prim_center['slc_pfp_n_hits'] >= nHits]

    df_hnl_prim_edge = df_hnl_prim_edge[df_hnl_prim_edge['slc_pfp_n_hits'] >= nHits]
    df_nu_prim_edge = df_nu_prim_edge[df_nu_prim_edge['slc_pfp_n_hits'] >= nHits]
    df_cos_prim_edge = df_cos_prim_edge[df_cos_prim_edge['slc_pfp_n_hits'] >= nHits]
    
df_hnl_prim = pd.concat([df_hnl_prim_center, df_hnl_prim_edge])
df_nu_prim = pd.concat([df_nu_prim_center, df_nu_prim_edge])
df_cos_prim = pd.concat([df_cos_prim_center, df_cos_prim_edge])

In [ ]:
df_hnl = merge_df_prim(df_hnl, df_hnl_prim)

df_nu = merge_df_prim(df_nu, df_nu_prim)

df_cos = merge_df_prim(df_cos, df_cos_prim)

<h3>Save Eff/Purity</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("N Hits Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Make Beam Bucket - Post N Hits</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + " [ns]",
                     ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postNhits.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Multiplicity</h1>

<h3>Apply Cut</h3>

<h3>Save Eff/Purity</h3>

<h1>Muon Rejection</h1>

In [ ]:
when_hnl_center = (df_hnl['mod_t'] >= lb) &  (df_hnl['mod_t'] <= ub)
when_nu_center = (df_nu['mod_t'] >= lb) &  (df_nu['mod_t'] <= ub)
when_cos_center = (df_cos['mod_t'] >= lb) &  (df_cos['mod_t'] <= ub)

when_hnl_edge = (df_hnl['mod_t'] < lb) | (df_hnl['mod_t'] > ub)
when_nu_edge = (df_nu['mod_t'] < lb) | (df_nu['mod_t'] > ub)
when_cos_edge = (df_cos['mod_t'] < lb) | (df_cos['mod_t'] > ub)

df_hnl_center = df_hnl[when_hnl_center]
df_nu_center = df_nu[when_nu_center]
df_cos_center = df_cos[when_cos_center]

df_hnl_edge = df_hnl[when_hnl_edge]
df_nu_edge = df_nu[when_nu_edge]
df_cos_edge = df_cos[when_cos_edge]

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_razzled_muons', 
                    xmin = 0, xmax = 6, xnbin = 6,
                    xtitle = 'n Razzled Muons',
                     ifAddLegend = True, addLegend = label
                    )

plt.axvline(x = 1, c = 'r', linestyle = '--', lw = 1.5)
plt.annotate("", xy=(0.5, 8.5e6), xytext=(1, 8.5e6)
             , arrowprops=dict(arrowstyle= '-|>',color='r',lw=1.5,ls='--'))
plt.ylim(0, 9e6)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_muon_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_muon_score', 
                    xmin = 0, xmax = 1, xnbin = 50,
                    xtitle = 'PFP Razzled Muon Score',
                     ifAddLegend = True, addLegend = label
                    )
plt.axvline(x =  TuneCut['MuonPeak'], c = 'r', linestyle = '--', lw = 1.5)
plt.annotate("", xy=(0.0, 8.5e6), xytext=(TuneCut['MuonPeak'], 8.5e6)
             , arrowprops=dict(arrowstyle= '-|>',color='r',lw=1.5,ls='--'))
plt.ylim(0, 9e6)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_muon_score_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
plot_slc_var(df_hnl_center, df_nu_center, df_cos_center,
                    true_counts, start_counts, 
                    'slc_n_razzled_muons', 
                    xmin = 0, xmax = 6, xnbin = 6,
                    xtitle = 'n Razzled Muons',
                     ifAddLegend = True, addLegend = label
                    )

plt.axvline(x = 1, c = 'r', linestyle = '--', lw = 1.5)
plt.annotate("", xy=(0.5, 8.5e6), xytext=(1, 8.5e6)
             , arrowprops=dict(arrowstyle= '-|>',color='r',lw=1.5,ls='--'))
plt.ylim(0, 9e6)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_muon_peak_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl_center, df_nu_center, df_cos_center,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_muon_score', 
                    xmin = 0, xmax = 1, xnbin = 50,
                    xtitle = 'PFP Razzled Muon Score',
                     ifAddLegend = True, addLegend = label
                    )

plt.axvline(x =  TuneCut['MuonPeak'], c = 'r', linestyle = '--', lw = 1.5)
plt.annotate("", xy=(0.0, 8.5e6), xytext=(TuneCut['MuonPeak'], 8.5e6)
             , arrowprops=dict(arrowstyle= '-|>',color='r',lw=1.5,ls='--'))
plt.ylim(0, 9e6)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_muon_score_peak_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
plot_slc_var(df_hnl_edge, df_nu_edge, df_cos_edge,
                    true_counts, start_counts, 
                    'slc_n_razzled_muons', 
                    xmin = 0, xmax = 6, xnbin = 6,
                    xtitle = 'n Razzled Muons',
                     ifAddLegend = True, addLegend = label
                    )

plt.axvline(x = 1, c = 'r', linestyle = '--', lw = 1.5)
plt.annotate("", xy=(0.5, 3.2e5), xytext=(1, 3.2e5)
             , arrowprops=dict(arrowstyle= '-|>',color='r',lw=1.5,ls='--'))
plt.ylim(0, 3.3e5)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_muon_edge_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl_edge, df_nu_edge, df_cos_edge,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_muon_score', 
                    xmin = 0, xmax = 1, xnbin = 50,
                    xtitle = 'PFP Razzled Muon Score',
                     ifAddLegend = True, addLegend = label
                    )
plt.axvline(x =  TuneCut['MuonEdge'], c = 'r', linestyle = '--', lw = 1.5)
plt.annotate("", xy=(0.0, 3.2e5), xytext=(TuneCut['MuonEdge'], 3.2e5)
             , arrowprops=dict(arrowstyle= '-|>',color='r',lw=1.5,ls='--'))
plt.ylim(0, 3.3e5)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_muon_score_edge_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
if ifTune:
    vary_MuonScore(df_hnl_center, df_nu_center, df_cos_center , true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/muon_center_tuning.png", dpi =200)
        
    vary_MuonScore(df_hnl_edge, df_nu_edge, df_cos_edge , true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/muon_edge_tuning.png", dpi =200)

<h3> Apply Cut</h3>

In [ ]:
if ifCut:
    muonPeak = TuneCut['MuonPeak']
    print(muonPeak)
    df_hnl_center = cutMuon(df_hnl_center, muonScore =muonPeak)
    df_nu_center = cutMuon(df_nu_center, muonScore =muonPeak)
    df_cos_center = cutMuon(df_cos_center,muonScore =muonPeak)
    
    muonEdge = TuneCut['MuonEdge']
    print(muonEdge)
    df_hnl_edge = cutMuon(df_hnl_edge, muonScore =muonEdge)
    df_nu_edge = cutMuon(df_nu_edge, muonScore =muonEdge)
    df_cos_edge = cutMuon(df_cos_edge,muonScore =muonEdge)   
    
df_hnl = pd.concat([df_hnl_center, df_hnl_edge])
df_nu = pd.concat([df_nu_center, df_nu_edge])
df_cos = pd.concat([df_cos_center, df_cos_edge])

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Muon Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Make Beam Bucket - Post Muon</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + " [ns]",
                     ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postmuon.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Proton Rejection</h1>

In [ ]:
when_hnl_center = (df_hnl['mod_t'] >= lb) &  (df_hnl['mod_t'] <= ub)
when_nu_center = (df_nu['mod_t'] >= lb) &  (df_nu['mod_t'] <= ub)
when_cos_center = (df_cos['mod_t'] >= lb) &  (df_cos['mod_t'] <= ub)

when_hnl_edge = (df_hnl['mod_t'] < lb) | (df_hnl['mod_t'] > ub)
when_nu_edge = (df_nu['mod_t'] < lb) | (df_nu['mod_t'] > ub)
when_cos_edge = (df_cos['mod_t'] < lb) | (df_cos['mod_t'] > ub)

df_hnl_center = df_hnl[when_hnl_center]
df_nu_center = df_nu[when_nu_center]
df_cos_center = df_cos[when_cos_center]

df_hnl_edge = df_hnl[when_hnl_edge]
df_nu_edge = df_nu[when_nu_edge]
df_cos_edge = df_cos[when_cos_edge]

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_razzled_protons_thresh', 
                    xmin = 0, xmax = 6, xnbin = 6,
                    xtitle = 'n Razzled Protons',
                     ifAddLegend = True, addLegend = label
                    )

plt.axvline(x = 1, c = 'r', linestyle = '--', lw = 1.5)
plt.annotate("", xy=(0.5, 7.5e5), xytext=(1, 7.5e5)
             , arrowprops=dict(arrowstyle= '-|>',color='r',lw=1.5,ls='--'))
plt.ylim(0, 8e5)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_proton_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_proton_score', 
                    xmin = 0, xmax = 1, xnbin = 50,
                    xtitle = 'PFP Razzled Proton Score',
                      ifAddLegend = True, addLegend = label, LegendLoc = 'upper center'
                    )

plt.axvline(x =  TuneCut['ProtonPeak'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(TuneCut['ProtonPeak'], 5.5e5), xytext=(TuneCut['ProtonPeak'] - 0.1, 5.5e5)
             , arrowprops=dict(arrowstyle= '<|-',color='r',lw=1.5,ls='--'))
plt.ylim(0, 6e5)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_proton_score_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
plot_slc_var(df_hnl_center, df_nu_center, df_cos_center,
                    true_counts, start_counts, 
                    'slc_n_razzled_protons_thresh', 
                    xmin = 0, xmax = 6, xnbin = 6,
                    xtitle = 'n Razzled Protons',
                     ifAddLegend = True, addLegend = label
                    )

plt.axvline(x = 1, c = 'r', linestyle = '--', lw = 1.5)
plt.annotate("", xy=(0.5, 7.5e5), xytext=(1, 7.5e5)
             , arrowprops=dict(arrowstyle= '-|>',color='r',lw=1.5,ls='--'))
plt.ylim(0, 8e5)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_proton_peak_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl_center, df_nu_center, df_cos_center,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_proton_score', 
                    xmin = 0, xmax = 1, xnbin = 50,
                    xtitle = 'PFP Razzled Proton Score',
                      ifAddLegend = True, addLegend = label, LegendLoc = 'upper center'
                    )

plt.axvline(x =  TuneCut['ProtonPeak'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(TuneCut['ProtonPeak'], 5.5e5), xytext=(TuneCut['ProtonPeak'] - 0.1, 5.5e5)
             , arrowprops=dict(arrowstyle= '<|-',color='r',lw=1.5,ls='--'))
plt.ylim(0, 6e5)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_proton_score_peak_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
plot_slc_var(df_hnl_edge, df_nu_edge, df_cos_edge,
                    true_counts, start_counts, 
                    'slc_n_razzled_protons_thresh', 
                    xmin = 0, xmax = 6, xnbin = 6,
                    xtitle = 'n Razzled Protons',
                     ifAddLegend = True, addLegend = label
                    )

plt.axvline(x = 1, c = 'r', linestyle = '--', lw = 1.5)
plt.annotate("", xy=(0.5, 3.8e4), xytext=(1, 3.8e4)
             , arrowprops=dict(arrowstyle= '-|>',color='r',lw=1.5,ls='--'))
plt.ylim(0, 4e4)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_proton_edge_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl_edge, df_nu_edge, df_cos_edge,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_proton_score', 
                    xmin = 0, xmax = 1, xnbin = 50,
                    xtitle = 'PFP Razzled Proton Score',
                      ifAddLegend = True, addLegend = label, LegendLoc = 'upper center'
                    )

plt.axvline(x =  TuneCut['ProtonEdge'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(TuneCut['ProtonEdge'], 3.0e4), xytext=(TuneCut['ProtonEdge'] - 0.1, 3.0e4)
             , arrowprops=dict(arrowstyle= '<|-',color='r',lw=1.5,ls='--'))
plt.ylim(0, 3.2e4)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_proton_score_edge_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
if ifTune:
    vary_ProtonScore(df_hnl_center, df_nu_center, df_cos_center, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/proton_center_tuning.png", dpi =200)
        
    vary_ProtonScore(df_hnl_edge, df_nu_edge, df_cos_edge, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/proton_edge_tuning.png", dpi =200)

<h3>Apply Cut</h3>

In [ ]:
if ifCut:
    protonPeak = TuneCut['ProtonPeak']
    print(protonPeak)
    df_hnl_center = cutProton(df_hnl_center, nProton = 0, protonScore = protonPeak)
    df_nu_center = cutProton(df_nu_center, nProton = 0, protonScore = protonPeak)
    df_cos_center = cutProton(df_cos_center, nProton = 0, protonScore = protonPeak)
    
    protonEdge = TuneCut['ProtonEdge']
    print(protonEdge)
    df_hnl_edge = cutProton(df_hnl_edge, nProton = 0, protonScore = protonEdge)
    df_nu_edge = cutProton(df_nu_edge, nProton = 0, protonScore = protonEdge)
    df_cos_edge = cutProton(df_cos_edge, nProton = 0, protonScore = protonEdge)
    
df_hnl = pd.concat([df_hnl_center, df_hnl_edge])
df_nu = pd.concat([df_nu_center, df_nu_edge])
df_cos = pd.concat([df_cos_center, df_cos_edge])

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Proton Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Make Beam Bucket - Post Proton</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + " [ns]",
                 ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postproton.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Pion Rejection</h1>

In [ ]:
when_hnl_center = (df_hnl['mod_t'] >= lb) &  (df_hnl['mod_t'] <= ub)
when_nu_center = (df_nu['mod_t'] >= lb) &  (df_nu['mod_t'] <= ub)
when_cos_center = (df_cos['mod_t'] >= lb) &  (df_cos['mod_t'] <= ub)

when_hnl_edge = (df_hnl['mod_t'] < lb) | (df_hnl['mod_t'] > ub)
when_nu_edge = (df_nu['mod_t'] < lb) | (df_nu['mod_t'] > ub)
when_cos_edge = (df_cos['mod_t'] < lb) | (df_cos['mod_t'] > ub)

df_hnl_center = df_hnl[when_hnl_center]
df_nu_center = df_nu[when_nu_center]
df_cos_center = df_cos[when_cos_center]

df_hnl_edge = df_hnl[when_hnl_edge]
df_nu_edge = df_nu[when_nu_edge]
df_cos_edge = df_cos[when_cos_edge]

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_razzled_pions_thresh', 
                    xmin = 0, xmax = 6, xnbin = 6,
                    xtitle = 'n Razzled Pions',
                      ifAddLegend = True, addLegend = label
                    )

plt.axvline(x = 1, c = 'r', linestyle = '--', lw = 1.5)
plt.annotate("", xy=(1, 4.3e5), xytext=(0.5, 4.3e5)
, arrowprops=dict(arrowstyle= '<|-', color='r', lw=1.5, ls='--'))
plt.ylim(0, 4.5e5)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_pion_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_pion_score', 
                    xmin = 0, xmax = 1, xnbin = 50,
                    xtitle = 'PFP Razzled Pion Score',
                      ifAddLegend = True, addLegend = label, LegendLoc = 'upper center'
                    )

plt.axvline(x =  TuneCut['PionPeak'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(TuneCut['PionPeak'], 1.7e5), xytext=(TuneCut['PionPeak'] - 0.1, 1.7e5)
, arrowprops=dict(arrowstyle= '<|-', color='r', lw=1.5, ls='--'))
plt.ylim(0, 1.8e5)

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_pion_score_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
plot_slc_var(df_hnl_center, df_nu_center, df_cos_center,
                    true_counts, start_counts, 
                    'slc_n_razzled_pions_thresh', 
                    xmin = 0, xmax = 6, xnbin = 6,
                    xtitle = 'n Razzled Pions',
                      ifAddLegend = True, addLegend = label
                    )

plt.axvline(x = 1, c = 'r', linestyle = '--', lw = 1.5)
plt.annotate("", xy=(1, 4.3e5), xytext=(0.5, 4.3e5)
, arrowprops=dict(arrowstyle= '<|-', color='r', lw=1.5, ls='--'))
plt.ylim(0, 4.5e5)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_pion_center_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl_center, df_nu_center, df_cos_center,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_pion_score', 
                    xmin = 0, xmax = 1, xnbin = 50,
                    xtitle = 'PFP Razzled Pion Score',
                      ifAddLegend = True, addLegend = label, LegendLoc = 'upper center'
                    )

plt.axvline(x =  TuneCut['PionPeak'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(TuneCut['PionPeak'], 1.7e5), xytext=(TuneCut['PionPeak'] - 0.1, 1.7e5)
, arrowprops=dict(arrowstyle= '<|-', color='r', lw=1.5, ls='--'))
plt.ylim(0, 1.8e5)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_pion_score_peak_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
plot_slc_var(df_hnl_edge, df_nu_edge, df_cos_edge,
                    true_counts, start_counts, 
                    'slc_n_razzled_pions_thresh', 
                    xmin = 0, xmax = 6, xnbin = 6,
                    xtitle = 'n Razzled Pions',
                      ifAddLegend = True, addLegend = label
                    )

plt.axvline(x = 1, c = 'r', linestyle = '--', lw = 1.5)
plt.annotate("", xy=(1, 3.3e4), xytext=(0.5, 3.3e4)
, arrowprops=dict(arrowstyle= '<|-', color='r', lw=1.5, ls='--'))
plt.ylim(0, 3.4e4)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_pion_edge_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl_edge, df_nu_edge, df_cos_edge,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_pion_score', 
                    xmin = 0, xmax = 1, xnbin = 50,
                    xtitle = 'PFP Razzled Pion Score',
                      ifAddLegend = True, addLegend = label, LegendLoc = 'upper center'
                    )

plt.axvline(x =  TuneCut['PionEdge'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(TuneCut['PionEdge'], 1.1e4), xytext=(TuneCut['PionEdge'] - 0.1, 1.1e4)
, arrowprops=dict(arrowstyle= '<|-', color='r', lw=1.5, ls='--'))
plt.ylim(0, 1.2e4)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_pion_score_edge_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
if ifTune:
    vary_PionScore(df_hnl_center, df_nu_center, df_cos_center, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/pion_center_tuning.png", dpi =200)
        
    vary_PionScore(df_hnl_edge, df_nu_edge, df_cos_edge, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/pion_edge_tuning.png", dpi =200)

<h3>Apply Cut </h3>

In [ ]:
if ifCut:
    pionPeak = TuneCut['PionPeak']
    print(pionPeak)
    df_hnl_center = cutPion(df_hnl_center, nPion = 0, pionScore = pionPeak)
    df_nu_center = cutPion(df_nu_center, nPion = 0, pionScore = pionPeak)
    df_cos_center = cutPion(df_cos_center, nPion = 0, pionScore = pionPeak)
    
    pionEdge = TuneCut['PionEdge']
    print(pionEdge)
    df_hnl_edge = cutPion(df_hnl_edge, nPion = 0, pionScore = pionEdge)
    df_nu_edge = cutPion(df_nu_edge, nPion = 0, pionScore = pionEdge)
    df_cos_edge = cutPion(df_cos_edge, nPion = 0, pionScore = pionEdge)
    
df_hnl = pd.concat([df_hnl_center, df_hnl_edge])
df_nu = pd.concat([df_nu_center, df_nu_edge])
df_cos = pd.concat([df_cos_center, df_cos_edge])

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Pion Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Make Beam Bucket - Post Pion</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + " [ns]",
                     ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postpion.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Electron Rejection</h1>

<h3> Get Primary Shower </h3>

In [ ]:
df_hnl_prim = get_primary_shw_df(df_hnl)

df_nu_prim = get_primary_shw_df(df_nu)

df_cos_prim = get_primary_shw_df(df_cos)

<h3>Split into Peak/Edge</h3>

In [ ]:
when_hnl_center = (df_hnl_prim['mod_t'] >= lb) &  (df_hnl_prim['mod_t'] <= ub)
when_nu_center = (df_nu_prim['mod_t'] >= lb) &  (df_nu_prim['mod_t'] <= ub)
when_cos_center = (df_cos_prim['mod_t'] >= lb) &  (df_cos_prim['mod_t'] <= ub)

when_hnl_edge = (df_hnl_prim['mod_t'] < lb) | (df_hnl_prim['mod_t'] > ub)
when_nu_edge = (df_nu_prim['mod_t'] < lb) | (df_nu_prim['mod_t'] > ub)
when_cos_edge = (df_cos_prim['mod_t'] < lb) | (df_cos_prim['mod_t'] > ub)

df_hnl_prim_center = df_hnl_prim[when_hnl_center]
df_nu_prim_center = df_nu_prim[when_nu_center]
df_cos_prim_center = df_cos_prim[when_cos_center]

df_hnl_prim_edge = df_hnl_prim[when_hnl_edge]
df_nu_prim_edge = df_nu_prim[when_nu_edge]
df_cos_prim_edge = df_cos_prim[when_cos_edge]

<h3> Primary Shower </h3>

In [ ]:
#----------------------------------------------
plot_slc_var(df_hnl_prim
            , df_nu_prim
             , df_cos_prim
                    ,true_counts, start_counts, 
                    'slc_pfp_razzled_electron_score', 
                    xmin = 0, xmax = 1, xnbin = 50,
                    xtitle = 'PFP Razzled Electron Score',
                  ifAddLegend = True, addLegend = label, LegendLoc ='upper center'
                    )

plt.axvline(x =  TuneCut['ElectronPeak'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(TuneCut['ElectronPeak'], 8.5e3), xytext=(TuneCut['ElectronPeak'] - 0.1, 8.5e3)
             , arrowprops=dict(arrowstyle= '<|-',color='r',lw=1.5,ls='--'))
plt.ylim(0, 1e4)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_electron_score_prim_shw_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
#----------------------------------------------
plot_slc_var(df_hnl_prim_center
            , df_nu_prim_center
             , df_cos_prim_center,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_electron_score', 
                    xmin = 0, xmax = 1, xnbin = 50,
                    xtitle = 'PFP Razzled Electron Score',
                  ifAddLegend = True, addLegend = label, LegendLoc ='upper center'
                    )

plt.axvline(x =  TuneCut['ElectronPeak'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(TuneCut['ElectronPeak'], 8.5e3), xytext=(TuneCut['ElectronPeak'] - 0.1, 8.5e3)
             , arrowprops=dict(arrowstyle= '<|-',color='r',lw=1.5,ls='--'))
plt.ylim(0, 1e4)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_electron_score_prim_shw_peak_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
#----------------------------------------------
plot_slc_var(df_hnl_prim_edge
            , df_nu_prim_edge
             , df_cos_prim_edge,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_electron_score', 
                    xmin = 0, xmax = 1, xnbin = 50,
                    xtitle = 'PFP Razzled Electron Score',
                  ifAddLegend = True, addLegend = label, LegendLoc ='upper center'
                    )

plt.axvline(x =  TuneCut['ElectronEdge'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(TuneCut['ElectronEdge'], 1.1e3), xytext=(TuneCut['ElectronEdge'] - 0.1, 1.1e3)
             , arrowprops=dict(arrowstyle= '<|-',color='r',lw=1.5,ls='--'))
plt.ylim(0, 1.2e3)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_electron_score_prim_shw_edge_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3> Apply Cut </h3>

In [ ]:
if ifCut:
    electronPeak = TuneCut['ElectronPeak']
    print(electronPeak)
    df_hnl_prim_center = cutElectronLess(df_hnl_prim_center, ifnElectron = False, electronScore = electronPeak)
    df_nu_prim_center = cutElectronLess(df_nu_prim_center, ifnElectron = False, electronScore = electronPeak)
    df_cos_prim_center = cutElectronLess(df_cos_prim_center, ifnElectron = False, electronScore = electronPeak)
    
    electronEdge = TuneCut['ElectronEdge']
    print(electronEdge)
    df_hnl_prim_edge = cutElectronLess(df_hnl_prim_edge, ifnElectron = False, electronScore = electronEdge)
    df_nu_prim_edge = cutElectronLess(df_nu_prim_edge, ifnElectron = False, electronScore = electronEdge)
    df_cos_prim_edge = cutElectronLess(df_cos_prim_edge, ifnElectron = False, electronScore = electronEdge)
    
df_hnl_prim = pd.concat([df_hnl_prim_center, df_hnl_prim_edge])
df_nu_prim = pd.concat([df_nu_prim_center, df_nu_prim_edge])
df_cos_prim = pd.concat([df_cos_prim_center, df_cos_prim_edge])

In [ ]:
df_hnl = merge_df_prim(df_hnl, df_hnl_prim)

df_nu = merge_df_prim(df_nu, df_nu_prim)

df_cos = merge_df_prim(df_cos, df_cos_prim)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Electron Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Make Beam Bucket - Post Electron</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + " [ns]",
                     ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postelectron.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>PFP Track Score</h1>

<h3>Get Primary Shower</h3>

In [ ]:
df_hnl_prim = get_primary_shw_df(df_hnl)

df_nu_prim = get_primary_shw_df(df_nu)

df_cos_prim = get_primary_shw_df(df_cos)

<h3>Split into 1 and 2+ shower cases </h3>

In [ ]:
df_hnl_shw1, df_hnl_shw2 = split_my_df(df_hnl)

df_nu_shw1, df_nu_shw2 = split_my_df(df_nu)

df_cos_shw1, df_cos_shw2 = split_my_df(df_cos)

<h3> One Shower Case : Peak vs Edge Bucket </h3>

In [ ]:
when_hnl_shw1_center = (df_hnl_shw1['mod_t'] >= lb) &  (df_hnl_shw1['mod_t'] <= ub)
when_nu_shw1_center = (df_nu_shw1['mod_t'] >= lb) &  (df_nu_shw1['mod_t'] <= ub)
when_cos_shw1_center = (df_cos_shw1['mod_t'] >= lb) &  (df_cos_shw1['mod_t'] <= ub)

when_hnl_shw1_edge = (df_hnl_shw1['mod_t'] < lb) | (df_hnl_shw1['mod_t'] > ub)
when_nu_shw1_edge = (df_nu_shw1['mod_t'] < lb) | (df_nu_shw1['mod_t'] > ub)
when_cos_shw1_edge = (df_cos_shw1['mod_t'] < lb) | (df_cos_shw1['mod_t'] > ub)

df_hnl_shw1_center = df_hnl_shw1[when_hnl_shw1_center]
df_nu_shw1_center = df_nu_shw1[when_nu_shw1_center]
df_cos_shw1_center = df_cos_shw1[when_cos_shw1_center]

df_hnl_shw1_edge = df_hnl_shw1[when_hnl_shw1_edge]
df_nu_shw1_edge = df_nu_shw1[when_nu_shw1_edge]
df_cos_shw1_edge = df_cos_shw1[when_cos_shw1_edge]

In [ ]:
plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos_shw1,
                    true_counts, start_counts, 
                    'slc_pfp_track_score', 
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Track Score',
                  ifAddLegend = True, addLegend = label
                    )
plt.axvline(x =  TuneCut['OneShowerCenterTrackLess'], c = 'r', linestyle = '--')
plt.axvline(x =  TuneCut['OneShowerTrackMore'], c = 'r', linestyle = '--')

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"one_shw_track_score.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
plot_slc_var(df_hnl_shw1_center, df_nu_shw1_center, df_cos_shw1_center,
                    true_counts, start_counts, 
                    'slc_pfp_track_score', 
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Track Score',
                  ifAddLegend = True, addLegend = label
                    )
plt.axvline(x =  TuneCut['OneShowerCenterTrackLess'], c = 'r', linestyle = '--')
plt.axvline(x =  TuneCut['OneShowerTrackMore'], c = 'r', linestyle = '--')

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"one_shw_track_score_peak.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
plot_slc_var(df_hnl_shw1_edge, df_nu_shw1_edge, df_cos_shw1_edge,
                    true_counts, start_counts, 
                    'slc_pfp_track_score', 
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Track Score',
                  ifAddLegend = True, addLegend = label
                    )

plt.axvline(x =  TuneCut['OneShowerEdgeTrackLess'], c = 'r', linestyle = '--')
plt.axvline(x =  TuneCut['OneShowerTrackMore'], c = 'r', linestyle = '--')

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"one_shw_track_score_edge.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3>Cut on One Shower</h3>

In [ ]:
if ifCut:

    trk_lb = TuneCut['OneShowerTrackMore']
    trk_ub = TuneCut['OneShowerCenterTrackLess']
    print(trk_ub)
    df_hnl_shw1_center = df_hnl_shw1_center[(df_hnl_shw1_center['slc_pfp_track_score'] >= trk_lb) &(df_hnl_shw1_center['slc_pfp_track_score'] <= trk_ub)]
    df_nu_shw1_center = df_nu_shw1_center[(df_nu_shw1_center['slc_pfp_track_score'] >= trk_lb) &(df_nu_shw1_center['slc_pfp_track_score'] <= trk_ub)]
    df_cos_shw1_center = df_cos_shw1_center[(df_cos_shw1_center['slc_pfp_track_score'] >= trk_lb) &(df_cos_shw1_center['slc_pfp_track_score'] <= trk_ub)]

    trk_lb = TuneCut['OneShowerTrackMore']
    trk_ub = TuneCut['OneShowerEdgeTrackLess']
    print(trk_ub)
    df_hnl_shw1_edge = df_hnl_shw1_edge[(df_hnl_shw1_edge['slc_pfp_track_score'] >= trk_lb) &(df_hnl_shw1_edge['slc_pfp_track_score'] <= trk_ub)]
    df_nu_shw1_edge = df_nu_shw1_edge[(df_nu_shw1_edge['slc_pfp_track_score'] >= trk_lb) &(df_nu_shw1_edge['slc_pfp_track_score'] <= trk_ub)]
    df_cos_shw1_edge = df_cos_shw1_edge[(df_cos_shw1_edge['slc_pfp_track_score'] >= trk_lb) &(df_cos_shw1_edge['slc_pfp_track_score'] <= trk_ub)]
    
df_hnl_shw1 = pd.concat([df_hnl_shw1_center, df_hnl_shw1_edge])
df_nu_shw1 = pd.concat([df_nu_shw1_center, df_nu_shw1_edge])
df_cos_shw1 = pd.concat([df_cos_shw1_center, df_cos_shw1_edge])

<h3>Primary of Two shower case</h3>

In [ ]:
df_nu_shw2 = df_nu_shw2.apply(pd.Series.explode).reset_index()
df_hnl_shw2 = df_hnl_shw2.apply(pd.Series.explode).reset_index()
df_cos_shw2 = df_cos_shw2.apply(pd.Series.explode).reset_index()

df_hnl_prim_shw2 = get_primary_shw_df(df_hnl_shw2)

df_nu_prim_shw2 = get_primary_shw_df(df_nu_shw2)

df_cos_prim_shw2 = get_primary_shw_df(df_cos_shw2)

<h3>Two Showers: Split Peak vs Edge</h3>

In [ ]:
when_hnl_shw2_center = (df_hnl_prim_shw2['mod_t'] >= lb) &  (df_hnl_prim_shw2['mod_t'] <= ub)
when_nu_shw2_center = (df_nu_prim_shw2['mod_t'] >= lb) &  (df_nu_prim_shw2['mod_t'] <= ub)
when_cos_shw2_center = (df_cos_prim_shw2['mod_t'] >= lb) &  (df_cos_prim_shw2['mod_t'] <= ub)

when_hnl_shw2_edge = (df_hnl_prim_shw2['mod_t'] < lb) | (df_hnl_prim_shw2['mod_t'] > ub)
when_nu_shw2_edge = (df_nu_prim_shw2['mod_t'] < lb) | (df_nu_prim_shw2['mod_t'] > ub)
when_cos_shw2_edge = (df_cos_prim_shw2['mod_t'] < lb) | (df_cos_prim_shw2['mod_t'] > ub)

df_hnl_prim_shw2_center = df_hnl_prim_shw2[when_hnl_shw2_center]
df_nu_prim_shw2_center = df_nu_prim_shw2[when_nu_shw2_center]
df_cos_prim_shw2_center = df_cos_prim_shw2[when_cos_shw2_center]


df_hnl_prim_shw2_edge = df_hnl_prim_shw2[when_hnl_shw2_edge]
df_nu_prim_shw2_edge = df_nu_prim_shw2[when_nu_shw2_edge]
df_cos_prim_shw2_edge = df_cos_prim_shw2[when_cos_shw2_edge]

In [ ]:
plot_slc_var(df_hnl_prim_shw2, df_nu_prim_shw2, df_cos_prim_shw2,
                    true_counts, start_counts, 
                    'slc_pfp_track_score', 
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Track Score',
                  ifAddLegend = True, addLegend = label
                    )
plt.axvline(x =  TuneCut['TwoShowerCenterTrackLess'], c = 'r', linestyle = '--')
plt.axvline(x =  TuneCut['TwoShowerTrackMore'], c = 'r', linestyle = '--')

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"two_shower_primary_track_score_precut.png", dpi = 200)
#-----------------------------------#
plt.show()


In [ ]:
plot_slc_var(df_hnl_prim_shw2_center, df_nu_prim_shw2_center, df_cos_prim_shw2_center,
                    true_counts, start_counts, 
                    'slc_pfp_track_score', 
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Track Score',
                  ifAddLegend = True, addLegend = label
                    )
plt.axvline(x =  TuneCut['TwoShowerCenterTrackLess'], c = 'r', linestyle = '--')
plt.axvline(x =  TuneCut['TwoShowerTrackMore'], c = 'r', linestyle = '--')

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"two_shower_primary_track_score_peak_precut.png", dpi = 200)
#-----------------------------------#
plt.show()


In [ ]:
plot_slc_var(df_hnl_prim_shw2_edge, df_nu_prim_shw2_edge, df_cos_prim_shw2_edge,
                    true_counts, start_counts, 
                    'slc_pfp_track_score', 
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Track Score',
                  ifAddLegend = True, addLegend = label
                    )
plt.axvline(x =  TuneCut['TwoShowerEdgeTrackLess'], c = 'r', linestyle = '--')
plt.axvline(x =  TuneCut['TwoShowerTrackMore'], c = 'r', linestyle = '--')

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"two_shower_primary_track_score_edge_precut.png", dpi = 200)
#-----------------------------------#
plt.show()


<h3>Cut on Two Shower</h3>

In [ ]:
if ifCut:
    
    trk_lb = TuneCut['TwoShowerTrackMore']
    trk_ub = TuneCut['TwoShowerCenterTrackLess']
    print(trk_ub)
    df_hnl_prim_shw2_center = df_hnl_prim_shw2_center[(df_hnl_prim_shw2_center['slc_pfp_track_score'] >= trk_lb) &(df_hnl_prim_shw2_center['slc_pfp_track_score'] <= trk_ub)]
    df_nu_prim_shw2_center = df_nu_prim_shw2_center[(df_nu_prim_shw2_center['slc_pfp_track_score'] >= trk_lb) &(df_nu_prim_shw2_center['slc_pfp_track_score'] <= trk_ub)]
    df_cos_prim_shw2_center = df_cos_prim_shw2_center[(df_cos_prim_shw2_center['slc_pfp_track_score'] >= trk_lb) &(df_cos_prim_shw2_center['slc_pfp_track_score'] <= trk_ub)]
    
    trk_lb = TuneCut['TwoShowerTrackMore']
    trk_ub = TuneCut['TwoShowerEdgeTrackLess']
    print(trk_ub)
    df_hnl_prim_shw2_edge = df_hnl_prim_shw2_edge[(df_hnl_prim_shw2_edge['slc_pfp_track_score'] >= trk_lb) &(df_hnl_prim_shw2_edge['slc_pfp_track_score'] <= trk_ub)]
    df_nu_prim_shw2_edge = df_nu_prim_shw2_edge[(df_nu_prim_shw2_edge['slc_pfp_track_score'] >= trk_lb) &(df_nu_prim_shw2_edge['slc_pfp_track_score'] <= trk_ub)]
    df_cos_prim_shw2_edge = df_cos_prim_shw2_edge[(df_cos_prim_shw2_edge['slc_pfp_track_score'] >= trk_lb) &(df_cos_prim_shw2_edge['slc_pfp_track_score'] <= trk_ub)]

    
df_hnl_prim_shw2 = pd.concat([df_hnl_prim_shw2_center, df_hnl_prim_shw2_edge])
df_nu_prim_shw2 = pd.concat([df_nu_prim_shw2_center, df_nu_prim_shw2_edge])
df_cos_prim_shw2 = pd.concat([df_cos_prim_shw2_center, df_cos_prim_shw2_edge])

<h3>Merge 1 and 2+ Showers </h3>

In [ ]:
df_hnl_shw2 = merge_df_prim(df_hnl_shw2, df_hnl_prim_shw2)
df_nu_shw2 = merge_df_prim(df_nu_shw2, df_nu_prim_shw2)
df_cos_shw2 = merge_df_prim(df_cos_shw2, df_cos_prim_shw2)

df_hnl = apply_shower_cut(df_hnl, df_hnl_shw1, df_hnl_shw2)
df_nu = apply_shower_cut(df_nu, df_nu_shw1, df_nu_shw2)
df_cos = apply_shower_cut(df_cos, df_cos_shw1, df_cos_shw2)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Track Score Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1>Beam Bucket - Post Track Score </h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',
                  ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postrackscore.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Opt0 Frac </h1>

<h3> Split 1 and 2+ shower cases </h3>

In [ ]:
df_hnl_shw1, df_hnl_shw2 = split_my_df(df_hnl)

df_nu_shw1, df_nu_shw2 = split_my_df(df_nu)

df_cos_shw1, df_cos_shw2 = split_my_df(df_cos)

<h3> One Shower: Split Center vs Edge </h3>

In [ ]:
when_hnl_shw1_center = (df_hnl_shw1['mod_t'] >= lb) &  (df_hnl_shw1['mod_t'] <= ub)
when_nu_shw1_center = (df_nu_shw1['mod_t'] >= lb) &  (df_nu_shw1['mod_t'] <= ub)
when_cos_shw1_center = (df_cos_shw1['mod_t'] >= lb) &  (df_cos_shw1['mod_t'] <= ub)

when_hnl_shw1_edge = (df_hnl_shw1['mod_t'] < lb) | (df_hnl_shw1['mod_t'] > ub)
when_nu_shw1_edge = (df_nu_shw1['mod_t'] < lb) | (df_nu_shw1['mod_t'] > ub)
when_cos_shw1_edge = (df_cos_shw1['mod_t'] < lb) | (df_cos_shw1['mod_t'] > ub)

df_hnl_shw1_center = df_hnl_shw1[when_hnl_shw1_center]
df_nu_shw1_center = df_nu_shw1[when_nu_shw1_center]
df_cos_shw1_center = df_cos_shw1[when_cos_shw1_center]


df_hnl_shw1_edge = df_hnl_shw1[when_hnl_shw1_edge]
df_nu_shw1_edge = df_nu_shw1[when_nu_shw1_edge]
df_cos_shw1_edge = df_cos_shw1[when_cos_shw1_edge]

In [ ]:
plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos_shw1,
                    true_counts, start_counts, 
                    'slc_opt0_frac', 
                    xmin = -1, xmax = 2, xnbin = 50,
                    #xmin = -0.5, xmax = 0.5, xnbin = 50,
                    xtitle = 'Opt0 Fraction',
                  ifAddLegend = True, addLegend = label
                    )
plt.axvline(x =  TuneCut['OneShowerCenterFracMore'], c = 'r', linestyle = '--')
plt.axvline(x =  TuneCut['OneShowerFracLess'], c = 'r', linestyle = '--')

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"opt0frac_one_shw_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

print( TuneCut['OneShowerCenterFracMore'])

In [ ]:
plot_slc_var(df_hnl_shw1_center, df_nu_shw1_center, df_cos_shw1_center,
                    true_counts, start_counts, 
                    'slc_opt0_frac', 
                    xmin = -1, xmax = 2, xnbin = 50,
                    #xmin = -0.5, xmax = 0.5, xnbin = 50,
                    xtitle = 'Opt0 Fraction',
                  ifAddLegend = True, addLegend = label
                    )
plt.axvline(x =  TuneCut['OneShowerCenterFracMore'], c = 'r', linestyle = '--')
plt.axvline(x =  TuneCut['OneShowerFracLess'], c = 'r', linestyle = '--')

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"opt0frac_one_shw_peak_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

print( TuneCut['OneShowerCenterFracMore'])

In [ ]:
plot_slc_var(df_hnl_shw1_edge, df_nu_shw1_edge, df_cos_shw1_edge,
                    true_counts, start_counts, 
                    'slc_opt0_frac', 
                    xmin = -1, xmax = 2, xnbin = 50,
                    #xmin = -0.5, xmax = 0.5, xnbin = 50,
                    xtitle = 'Opt0 Fraction',
                  ifAddLegend = True, addLegend = label
                    )
plt.axvline(x =  TuneCut['OneShowerEdgeFracMore'], c = 'r', linestyle = '--')
plt.axvline(x =  TuneCut['OneShowerFracLess'], c = 'r', linestyle = '--')

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"opt0frac_one_shw_edge_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

print( TuneCut['OneShowerEdgeFracMore'])

In [ ]:
if ifTune:
    vary_OpT0FracMore(df_hnl_shw1_center, df_nu_shw1_center, df_cos_shw1_center, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/opt0frac_more_one_shw_center_tuning.png", dpi =200)
        
    vary_OpT0FracMore(df_hnl_shw1_edge, df_nu_shw1_edge, df_cos_shw1_edge, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/opt0frac_more_one_shw_edge_tuning.png", dpi =200)

<h3>Apply Cut 1 shower only: harsh cut</h3>

In [ ]:
if ifCut:
    fracMore = TuneCut['OneShowerCenterFracMore']
    fracLess = TuneCut['OneShowerFracLess']
    print(fracMore)
    
    df_hnl_shw1_center = cutOpt0Frac(df_hnl_shw1_center, Opt0FracMore = fracMore, Opt0FracLess = fracLess)
    df_nu_shw1_center = cutOpt0Frac(df_nu_shw1_center, Opt0FracMore = fracMore, Opt0FracLess = fracLess)
    df_cos_shw1_center = cutOpt0Frac(df_cos_shw1_center, Opt0FracMore = fracMore, Opt0FracLess = fracLess)
    
    fracMore = TuneCut['OneShowerEdgeFracMore']
    fracLess = TuneCut['OneShowerFracLess']
    print(fracMore)
    
    df_hnl_shw1_edge = cutOpt0Frac(df_hnl_shw1_edge, Opt0FracMore = fracMore, Opt0FracLess = fracLess)
    df_nu_shw1_edge = cutOpt0Frac(df_nu_shw1_edge, Opt0FracMore = fracMore, Opt0FracLess = fracLess)
    df_cos_shw1_edge = cutOpt0Frac(df_cos_shw1_edge, Opt0FracMore = fracMore, Opt0FracLess = fracLess)
    
df_hnl_shw1 = pd.concat([df_hnl_shw1_center, df_hnl_shw1_edge])
df_nu_shw1 = pd.concat([df_nu_shw1_center, df_nu_shw1_edge])
df_cos_shw1 = pd.concat([df_cos_shw1_center, df_cos_shw1_edge])

<h3> Two+ Showers: Split Center vs Edge </h3>

In [ ]:
when_hnl_shw2_center = (df_hnl_shw2['mod_t'] >= lb) &  (df_hnl_shw2['mod_t'] <= ub)
when_nu_shw2_center = (df_nu_shw2['mod_t'] >= lb) &  (df_nu_shw2['mod_t'] <= ub)
when_cos_shw2_center = (df_cos_shw2['mod_t'] >= lb) &  (df_cos_shw2['mod_t'] <= ub)

when_hnl_shw2_edge = (df_hnl_shw2['mod_t'] < lb) | (df_hnl_shw2['mod_t'] > ub)
when_nu_shw2_edge = (df_nu_shw2['mod_t'] < lb) | (df_nu_shw2['mod_t'] > ub)
when_cos_shw2_edge = (df_cos_shw2['mod_t'] < lb) | (df_cos_shw2['mod_t'] > ub)

df_hnl_shw2_center = df_hnl_shw2[when_hnl_shw2_center]
df_nu_shw2_center = df_nu_shw2[when_nu_shw2_center]
df_cos_shw2_center = df_cos_shw2[when_cos_shw2_center]

df_hnl_shw2_edge = df_hnl_shw2[when_hnl_shw2_edge]
df_nu_shw2_edge = df_nu_shw2[when_nu_shw2_edge]
df_cos_shw2_edge = df_cos_shw2[when_cos_shw2_edge]

In [ ]:
plot_slc_var(df_hnl_shw2, df_nu_shw2, df_cos_shw2,
                    true_counts, start_counts, 
                    'slc_opt0_frac',
                    xmin = -1, xmax = 2, xnbin = 50,
                    #xmin = -0.5, xmax = 0.5, xnbin = 50,
                    xtitle = 'Opt0 Fraction',
                  ifAddLegend = True, addLegend = label
                    )
plt.axvline(x =  TuneCut['TwoShowerCenterFracMore'], c = 'r', linestyle = '--')
plt.axvline(x =  TuneCut['TwoShowerFracLess'], c = 'r', linestyle = '--')
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"opt0frac_two_shw_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

print( TuneCut['TwoShowerCenterFracMore'])

In [ ]:
plot_slc_var(df_hnl_shw2_center, df_nu_shw2_center, df_cos_shw2_center,
                    true_counts, start_counts, 
                    'slc_opt0_frac',
                    xmin = -1, xmax = 2, xnbin = 50,
                    #xmin = -0.5, xmax = 0.5, xnbin = 50,
                    xtitle = 'Opt0 Fraction',
                  ifAddLegend = True, addLegend = label
                    )
plt.axvline(x =  TuneCut['TwoShowerCenterFracMore'], c = 'r', linestyle = '--')
plt.axvline(x =  TuneCut['TwoShowerFracLess'], c = 'r', linestyle = '--')
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"opt0frac_two_shw_peak_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

print( TuneCut['TwoShowerCenterFracMore'])

In [ ]:
plot_slc_var(df_hnl_shw2_edge, df_nu_shw2_edge, df_cos_shw2_edge,
                    true_counts, start_counts, 
                    'slc_opt0_frac', 
                    xmin = -1, xmax = 2, xnbin = 50,
                    #xmin = -0.5, xmax = 0.5, xnbin = 50,
                    xtitle = 'Opt0 Fraction',
                  ifAddLegend = True, addLegend = label
                    )

plt.axvline(x =  TuneCut['TwoShowerEdgeFracMore'], c = 'r', linestyle = '--')
plt.axvline(x =  TuneCut['TwoShowerFracLess'], c = 'r', linestyle = '--')
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"opt0frac_two_shw_edge_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

print( TuneCut['TwoShowerEdgeFracMore'])

In [ ]:
if ifTune:
    vary_OpT0FracMore(df_hnl_shw2_center, df_nu_shw2_center, df_cos_shw2_center, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/opt0frac_more_two_shw_center_tuning.png", dpi =200)
        
    vary_OpT0FracMore(df_hnl_shw2_edge, df_nu_shw2_edge, df_cos_shw2_edge, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/opt0frac_more_two_shw_edge_tuning.png", dpi =200)

<h3> 2+ showers: cut on primary + less harsh cut </h3>

In [ ]:
if ifCut:
    fracMore = TuneCut['TwoShowerCenterFracMore']
    fracLess = TuneCut['TwoShowerFracLess']
    print(fracMore)
    
    df_hnl_shw2_center = cutOpt0Frac(df_hnl_shw2_center, Opt0FracMore = fracMore, Opt0FracLess = fracLess)
    df_nu_shw2_center = cutOpt0Frac(df_nu_shw2_center, Opt0FracMore = fracMore, Opt0FracLess = fracLess)
    df_cos_shw2_center = cutOpt0Frac(df_cos_shw2_center, Opt0FracMore = fracMore, Opt0FracLess = fracLess)
    
    fracMore = TuneCut['TwoShowerEdgeFracMore']
    fracLess = TuneCut['TwoShowerFracLess']
    print(fracMore)
    
    df_hnl_shw2_edge = cutOpt0Frac(df_hnl_shw2_edge, Opt0FracMore = fracMore, Opt0FracLess = fracLess)
    df_nu_shw2_edge = cutOpt0Frac(df_nu_shw2_edge, Opt0FracMore = fracMore, Opt0FracLess = fracLess)
    df_cos_shw2_edge = cutOpt0Frac(df_cos_shw2_edge, Opt0FracMore = fracMore, Opt0FracLess = fracLess)
    
df_hnl_shw2 = pd.concat([df_hnl_shw2_center, df_hnl_shw2_edge])
df_nu_shw2 = pd.concat([df_nu_shw2_center, df_nu_shw2_edge])
df_cos_shw2 = pd.concat([df_cos_shw2_center, df_cos_shw2_edge])

<h3> Merge 1 and 2 showers back </h3>

In [ ]:
df_hnl = apply_shower_cut(df_hnl, df_hnl_shw1, df_hnl_shw2)

df_nu = apply_shower_cut(df_nu, df_nu_shw1, df_nu_shw2)

df_cos = apply_shower_cut(df_cos, df_cos_shw1, df_cos_shw2)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Opt0 Fraction Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Make Beam Bucket - Post Opt0 Frac</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',
                  ifAddLegend = True, addLegend = label
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postopt0.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Shower Energy, dEdx and Conversion Gap</h1>

<h3> Get Primary Shower </h3>

In [ ]:
df_hnl_prim = get_primary_shw_df(df_hnl)

df_nu_prim = get_primary_shw_df(df_nu)

df_cos_prim = get_primary_shw_df(df_cos)

<h3> Split 1 and 2+ shower cases </h3>

In [ ]:
df_hnl_shw1, df_hnl_shw2 = split_my_df(df_hnl)

df_nu_shw1, df_nu_shw2 = split_my_df(df_nu)

df_cos_shw1, df_cos_shw2 = split_my_df(df_cos)

In [ ]:
print(len(df_hnl_prim))
print(len(df_nu_prim))
print(len(df_cos_prim))

print(len(df_hnl_shw1))
print(len(df_nu_shw1))
print(len(df_cos_shw1))

print(len(df_hnl_shw2))
print(len(df_nu_shw2))
print(len(df_cos_shw2))

<h3> One Shower: Split Center vs Edge </h3>

In [ ]:
when_hnl_shw1_center = (df_hnl_shw1['mod_t'] >= lb) &  (df_hnl_shw1['mod_t'] <= ub)
when_nu_shw1_center = (df_nu_shw1['mod_t'] >= lb) &  (df_nu_shw1['mod_t'] <= ub)
when_cos_shw1_center = (df_cos_shw1['mod_t'] >= lb) &  (df_cos_shw1['mod_t'] <= ub)

when_hnl_shw1_edge = (df_hnl_shw1['mod_t'] < lb) | (df_hnl_shw1['mod_t'] > ub)
when_nu_shw1_edge = (df_nu_shw1['mod_t'] < lb) | (df_nu_shw1['mod_t'] > ub)
when_cos_shw1_edge = (df_cos_shw1['mod_t'] < lb) | (df_cos_shw1['mod_t'] > ub)

df_hnl_shw1_center = df_hnl_shw1[when_hnl_shw1_center]
df_nu_shw1_center = df_nu_shw1[when_nu_shw1_center]
df_cos_shw1_center = df_cos_shw1[when_cos_shw1_center]

df_hnl_shw1_edge = df_hnl_shw1[when_hnl_shw1_edge]
df_nu_shw1_edge = df_nu_shw1[when_nu_shw1_edge]
df_cos_shw1_edge = df_cos_shw1[when_cos_shw1_edge]

In [ ]:
plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos_shw1,
                    true_counts, start_counts, 
                    'slc_pfp_shower_energy', 
                    xmin = 0, xmax = 2000, xnbin = 40,
                    xtitle = 'Shower Energy [MeV]',
                      ifAddLegend = True, addLegend = label
                         )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"prim_shower_energy_postpid_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

#================================================#
plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos_shw1,
                    true_counts, start_counts, 
                    'slc_pfp_shower_dedx', 
                    xmin = 0, xmax = 20, xnbin = 40,
                    xtitle = 'Shower dE/dx [MeV/cm]',
                      ifAddLegend = True, addLegend = label
                         )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"prim_shower_dedx_postpid_precut.png", dpi = 200)
#-----------------------------------#
plt.show()    
    
#================================================#
plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos_shw1,
                    true_counts, start_counts, 
                    'slc_pfp_shower_conv_gap', 
                    xmin = 0, xmax = 5, xnbin = 40,
                    xtitle = 'Shower Conversion Gap [cm]',
                      ifAddLegend = True, addLegend = label
                         )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"prim_shower_convgap_postpid_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
plot_slc_var(df_hnl_shw1_center, df_nu_shw1_center, df_cos_shw1_center,
#plot_slc_var(df_hnl_shw1_center, df_nu_shw1_center, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_energy', 
                    xmin = 0, xmax = 2000, xnbin = 40,
                    xtitle = 'Shower Energy [MeV]',
                      ifAddLegend = True, addLegend = label
                         )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"prim_shower_energy_postpid_peak_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

#================================================#
plot_slc_var(df_hnl_shw1_center, df_nu_shw1_center, df_cos_shw1_center,
#plot_slc_var(df_hnl_shw1_center, df_nu_shw1_center, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_dedx', 
                    xmin = 0, xmax = 20, xnbin = 40,
                    xtitle = 'Shower dE/dx [MeV/cm]',
                      ifAddLegend = True, addLegend = label
                         )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"prim_shower_dedx_postpid_peak_precut.png", dpi = 200)
#-----------------------------------#
plt.show()    
    
#================================================#
plot_slc_var(df_hnl_shw1_center, df_nu_shw1_center, df_cos_shw1_center,
#plot_slc_var(df_hnl_shw1_center, df_nu_shw1_center, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_conv_gap', 
                    xmin = 0, xmax = 5, xnbin = 40,
                    xtitle = 'Shower Conversion Gap [cm]',
                      ifAddLegend = True, addLegend = label
                         )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"prim_shower_convgap_postpid_peak_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
plot_slc_var(df_hnl_shw1_edge, df_nu_shw1_edge, df_cos_shw1_edge,
#plot_slc_var(df_hnl_shw1_edge, df_nu_shw1_edge, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_energy', 
                    xmin = 0, xmax = 2000, xnbin = 40,
                    xtitle = 'Shower Energy [MeV]',
                      ifAddLegend = True, addLegend = label
                         )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"prim_shower_energy_postpid_edge_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

#================================================#
plot_slc_var(df_hnl_shw1_edge, df_nu_shw1_edge, df_cos_shw1_edge,
#plot_slc_var(df_hnl_shw1_edge, df_nu_shw1_edge, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_dedx', 
                    xmin = 0, xmax = 20, xnbin = 40,
                    xtitle = 'Shower dE/dx [MeV/cm]',
                      ifAddLegend = True, addLegend = label
                         )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"prim_shower_dedx_postpid_edge_precut.png", dpi = 200)
#-----------------------------------#
plt.show()    
    
#================================================#
plot_slc_var(df_hnl_shw1_edge, df_nu_shw1_edge, df_cos_shw1_edge,
#plot_slc_var(df_hnl_shw1_edge, df_nu_shw1_edge, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_conv_gap', 
                    xmin = 0, xmax = 5, xnbin = 40,
                    xtitle = 'Shower Conversion Gap [cm]',
                      ifAddLegend = True, addLegend = label
                         )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"prim_shower_convgap_postpid_edge_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3> Primary Showers of 2+ shower cases </h3>

In [ ]:
df_nu_shw2 = df_nu_shw2.apply(pd.Series.explode).reset_index()
df_hnl_shw2 = df_hnl_shw2.apply(pd.Series.explode).reset_index()
df_cos_shw2 = df_cos_shw2.apply(pd.Series.explode).reset_index()

In [ ]:
df_hnl_prim_shw2 = get_primary_shw_df(df_hnl_shw2)

df_nu_prim_shw2 = get_primary_shw_df(df_nu_shw2)

df_cos_prim_shw2 = get_primary_shw_df(df_cos_shw2)

<h3> Two Showers: Split Center vs Edge </h3>

In [ ]:
when_hnl_shw2_center = (df_hnl_prim_shw2['mod_t'] >= lb) &  (df_hnl_prim_shw2['mod_t'] <= ub)
when_nu_shw2_center = (df_nu_prim_shw2['mod_t'] >= lb) &  (df_nu_prim_shw2['mod_t'] <= ub)
when_cos_shw2_center = (df_cos_prim_shw2['mod_t'] >= lb) &  (df_cos_prim_shw2['mod_t'] <= ub)

when_hnl_shw2_edge = (df_hnl_prim_shw2['mod_t'] < lb) | (df_hnl_prim_shw2['mod_t'] > ub)
when_nu_shw2_edge = (df_nu_prim_shw2['mod_t'] < lb) | (df_nu_prim_shw2['mod_t'] > ub)
when_cos_shw2_edge = (df_cos_prim_shw2['mod_t'] < lb) | (df_cos_prim_shw2['mod_t'] > ub)

df_hnl_prim_shw2_center = df_hnl_prim_shw2[when_hnl_shw2_center]
df_nu_prim_shw2_center = df_nu_prim_shw2[when_nu_shw2_center]
df_cos_prim_shw2_center = df_cos_prim_shw2[when_cos_shw2_center]


df_hnl_prim_shw2_edge = df_hnl_prim_shw2[when_hnl_shw2_edge]
df_nu_prim_shw2_edge = df_nu_prim_shw2[when_nu_shw2_edge]
df_cos_prim_shw2_edge = df_cos_prim_shw2[when_cos_shw2_edge]

In [ ]:
plot_slc_var(df_hnl_prim_shw2, df_nu_prim_shw2, df_cos_prim_shw2,
                    true_counts, start_counts, 
                    'slc_pfp_shower_energy', 
                    xmin = 0, xmax = 2000, xnbin = 40,
                    xtitle = 'Shower Energy [MeV]',
                      ifAddLegend = True, addLegend = label
                         )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"prim_two_shower_energy_postpid_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

#================================================#
plot_slc_var(df_hnl_prim_shw2, df_nu_prim_shw2_center, df_cos_prim_shw2,
                    true_counts, start_counts, 
                    'slc_pfp_shower_dedx', 
                    xmin = 0, xmax = 20, xnbin = 40,
                    xtitle = 'Shower dE/dx [MeV/cm]',
                      ifAddLegend = True, addLegend = label
                         )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"prim_two_shower_dedx_postpid_precut.png", dpi = 200)
#-----------------------------------#
plt.show()    
    
#================================================#
plot_slc_var(df_hnl_prim_shw2, df_nu_prim_shw2, df_cos_prim_shw2,
                    true_counts, start_counts, 
                    'slc_pfp_shower_conv_gap', 
                    xmin = 0, xmax = 5, xnbin = 40,
                    xtitle = 'Shower Conversion Gap [cm]',
                      ifAddLegend = True, addLegend = label
                         )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"prim_two_shower_convgap_postpid_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
plot_slc_var(df_hnl_prim_shw2_center, df_nu_prim_shw2_center, df_cos_prim_shw2_center,
#plot_slc_var(df_hnl_prim_shw2_center, df_nu_prim_shw2_center, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_energy', 
                    xmin = 0, xmax = 2000, xnbin = 40,
                    xtitle = 'Shower Energy [MeV]',
                      ifAddLegend = True, addLegend = label
                         )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"prim_two_shower_energy_postpid_peak_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

#================================================#
plot_slc_var(df_hnl_prim_shw2_center, df_nu_prim_shw2_center, df_cos_prim_shw2_center,
#plot_slc_var(df_hnl_prim_shw2_center, df_nu_prim_shw2_center, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_dedx', 
                    xmin = 0, xmax = 20, xnbin = 40,
                    xtitle = 'Shower dE/dx [MeV/cm]',
                      ifAddLegend = True, addLegend = label
                         )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"prim_two_shower_dedx_postpid_peak_precut.png", dpi = 200)
#-----------------------------------#
plt.show()    
    
#================================================#
plot_slc_var(df_hnl_prim_shw2_center, df_nu_prim_shw2_center, df_cos_prim_shw2_center,
#plot_slc_var(df_hnl_prim_shw2_center, df_nu_prim_shw2_center, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_conv_gap', 
                    xmin = 0, xmax = 5, xnbin = 40,
                    xtitle = 'Shower Conversion Gap [cm]',
                      ifAddLegend = True, addLegend = label
                         )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"prim_two_shower_convgap_postpid_peak_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
plot_slc_var(df_hnl_prim_shw2_edge, df_nu_prim_shw2_edge, df_cos_prim_shw2_edge,
#plot_slc_var(df_hnl_prim_shw2_edge, df_nu_prim_shw2_edge, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_energy', 
                    xmin = 0, xmax = 2000, xnbin = 40,
                    xtitle = 'Shower Energy [MeV]',
                      ifAddLegend = True, addLegend = label
                         )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"prim_two_shower_energy_postpid_edge_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

#================================================#
plot_slc_var(df_hnl_prim_shw2_edge, df_nu_prim_shw2_edge, df_cos_prim_shw2_edge,
#plot_slc_var(df_hnl_prim_shw2_edge, df_nu_prim_shw2_edge, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_dedx', 
                    xmin = 0, xmax = 20, xnbin = 40,
                    xtitle = 'Shower dE/dx [MeV/cm]',
                      ifAddLegend = True, addLegend = label
                         )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"prim_two_shower_dedx_postpid_edge_precut.png", dpi = 200)
#-----------------------------------#
plt.show()    
    
#================================================#
plot_slc_var(df_hnl_prim_shw2_edge, df_nu_prim_shw2_edge, df_cos_prim_shw2_edge,
#plot_slc_var(df_hnl_prim_shw2_edge, df_nu_prim_shw2_edge, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_conv_gap', 
                    xmin = 0, xmax = 5, xnbin = 40,
                    xtitle = 'Shower Conversion Gap [cm]',
                      ifAddLegend = True, addLegend = label
                         )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"prim_two_shower_convgap_postpid_edge_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Shower Theta</h1>

<h3> Get Primary Shower </h3>

In [ ]:
df_hnl_prim = get_primary_shw_df(df_hnl)

df_nu_prim = get_primary_shw_df(df_nu)

df_cos_prim = get_primary_shw_df(df_cos)

<h3> Split 1 and 2+ shower cases </h3>

In [ ]:
df_hnl_shw1, df_hnl_shw2 = split_my_df(df_hnl)

df_nu_shw1, df_nu_shw2 = split_my_df(df_nu)

df_cos_shw1, df_cos_shw2 = split_my_df(df_cos)

<h3> One Shower: Split Center vs Edge </h3>

In [ ]:
when_hnl_shw1_center = (df_hnl_shw1['mod_t'] >= lb) &  (df_hnl_shw1['mod_t'] <= ub)
when_nu_shw1_center = (df_nu_shw1['mod_t'] >= lb) &  (df_nu_shw1['mod_t'] <= ub)
when_cos_shw1_center = (df_cos_shw1['mod_t'] >= lb) &  (df_cos_shw1['mod_t'] <= ub)

when_hnl_shw1_edge = (df_hnl_shw1['mod_t'] < lb) | (df_hnl_shw1['mod_t'] > ub)
when_nu_shw1_edge = (df_nu_shw1['mod_t'] < lb) | (df_nu_shw1['mod_t'] > ub)
when_cos_shw1_edge = (df_cos_shw1['mod_t'] < lb) | (df_cos_shw1['mod_t'] > ub)

df_hnl_shw1_center = df_hnl_shw1[when_hnl_shw1_center]
df_nu_shw1_center = df_nu_shw1[when_nu_shw1_center]
df_cos_shw1_center = df_cos_shw1[when_cos_shw1_center]


df_hnl_shw1_edge = df_hnl_shw1[when_hnl_shw1_edge]
df_nu_shw1_edge = df_nu_shw1[when_nu_shw1_edge]
df_cos_shw1_edge = df_cos_shw1[when_cos_shw1_edge]

<h3> Primary Showers of 1 shower cases</h3>

In [ ]:
plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos_shw1,
#plot_slc_var(df_hnl_shw1_center, df_nu_shw1_center, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_theta', 
                    xmin = 0, xmax = 180, xnbin = 36,
                    xtitle = 'Shower Theta',
                  ifAddLegend = True, addLegend = label
                    )

plt.axvline(x =  TuneCut['OneShowerThetaCenter'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(TuneCut['OneShowerThetaCenter'], 1.5e3), xytext=(TuneCut['OneShowerThetaCenter'] - 15, 1.5e3)
             , arrowprops=dict(arrowstyle= '<|-',color='r',lw=1.5,ls='--'))
plt.ylim(0, 1.6e3)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"one_shower_theta_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

print(TuneCut['OneShowerThetaCenter'])

In [ ]:
plot_slc_var(df_hnl_shw1_center, df_nu_shw1_center, df_cos_shw1_center,
#plot_slc_var(df_hnl_shw1_center, df_nu_shw1_center, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_theta', 
                    xmin = 0, xmax = 180, xnbin = 36,
                    xtitle = 'Shower Theta',
                  ifAddLegend = True, addLegend = label
                    )

plt.axvline(x =  TuneCut['OneShowerThetaCenter'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(TuneCut['OneShowerThetaCenter'], 1.3e3), xytext=(TuneCut['OneShowerThetaCenter'] - 15, 1.3e3)
             , arrowprops=dict(arrowstyle= '<|-',color='r',lw=1.5,ls='--'))
plt.ylim(0, 1.4e3)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"one_shower_theta_peak_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

print(TuneCut['OneShowerThetaCenter'])

In [ ]:
plot_slc_var(df_hnl_shw1_edge, df_nu_shw1_edge, df_cos_shw1_edge,
#plot_slc_var(df_hnl_shw1_edge, df_nu_shw1_edge, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_theta', 
                    xmin = 0, xmax = 180, xnbin = 36,
                    xtitle = 'Shower Theta',
                  ifAddLegend = True, addLegend = label
                    )

plt.axvline(x =  TuneCut['OneShowerThetaEdge'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(TuneCut['OneShowerThetaEdge'], 2.3e2), xytext=(TuneCut['OneShowerThetaEdge'] - 15,2.3e2)
             , arrowprops=dict(arrowstyle= '<|-',color='r',lw=1.5,ls='--'))
plt.ylim(0, 2.4e2)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"one_shower_theta_edge_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

print(TuneCut['OneShowerThetaEdge'])

In [ ]:
if ifTune:
    vary_Theta(df_hnl_shw1_center, df_nu_shw1_center, df_cos_shw1_center, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/one_shower_prim_theta_center_tuning.png", dpi =200)
        
    vary_Theta(df_hnl_shw1_edge, df_nu_shw1_edge, df_cos_shw1_edge, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/one_shower_prim_theta_edge_tuning.png", dpi =200)

<h3> 1 shower only: harsh cut</h3>

In [ ]:
if ifCut:
    angle_center = TuneCut['OneShowerThetaCenter']
    df_hnl_shw1_center = cutThetaAngle(df_hnl_shw1_center, thetaAngle = angle_center) 
    df_nu_shw1_center = cutThetaAngle(df_nu_shw1_center, thetaAngle = angle_center) 
    df_cos_shw1_center = cutThetaAngle(df_cos_shw1_center, thetaAngle = angle_center) 

    angle_edge = TuneCut['OneShowerThetaEdge']
    df_hnl_shw1_edge = cutThetaAngle(df_hnl_shw1_edge, thetaAngle = angle_edge) 
    df_nu_shw1_edge = cutThetaAngle(df_nu_shw1_edge, thetaAngle = angle_edge) 
    df_cos_shw1_edge = cutThetaAngle(df_cos_shw1_edge, thetaAngle = angle_edge) 
    
df_hnl_shw1 = pd.concat([df_hnl_shw1_center, df_hnl_shw1_edge])
df_nu_shw1 = pd.concat([df_nu_shw1_center, df_nu_shw1_edge])
df_cos_shw1 = pd.concat([df_cos_shw1_center, df_cos_shw1_edge])

<h3> Primary Showers of 2+ shower cases </h3>

In [ ]:
df_nu_shw2 = df_nu_shw2.apply(pd.Series.explode).reset_index()
df_hnl_shw2 = df_hnl_shw2.apply(pd.Series.explode).reset_index()
df_cos_shw2 = df_cos_shw2.apply(pd.Series.explode).reset_index()

df_hnl_prim_shw2 = get_primary_shw_df(df_hnl_shw2)

df_nu_prim_shw2 = get_primary_shw_df(df_nu_shw2)

df_cos_prim_shw2 = get_primary_shw_df(df_cos_shw2)

<h3> Two Showers: Split Center vs Edge </h3>

In [ ]:
when_hnl_shw2_center = (df_hnl_prim_shw2['mod_t'] >= lb) &  (df_hnl_prim_shw2['mod_t'] <= ub)
when_nu_shw2_center = (df_nu_prim_shw2['mod_t'] >= lb) &  (df_nu_prim_shw2['mod_t'] <= ub)
when_cos_shw2_center = (df_cos_prim_shw2['mod_t'] >= lb) &  (df_cos_prim_shw2['mod_t'] <= ub)

when_hnl_shw2_edge = (df_hnl_prim_shw2['mod_t'] < lb) | (df_hnl_prim_shw2['mod_t'] > ub)
when_nu_shw2_edge = (df_nu_prim_shw2['mod_t'] < lb) | (df_nu_prim_shw2['mod_t'] > ub)
when_cos_shw2_edge = (df_cos_prim_shw2['mod_t'] < lb) | (df_cos_prim_shw2['mod_t'] > ub)

df_hnl_prim_shw2_center = df_hnl_prim_shw2[when_hnl_shw2_center]
df_nu_prim_shw2_center = df_nu_prim_shw2[when_nu_shw2_center]
df_cos_prim_shw2_center = df_cos_prim_shw2[when_cos_shw2_center]


df_hnl_prim_shw2_edge = df_hnl_prim_shw2[when_hnl_shw2_edge]
df_nu_prim_shw2_edge = df_nu_prim_shw2[when_nu_shw2_edge]
df_cos_prim_shw2_edge = df_cos_prim_shw2[when_cos_shw2_edge]

In [ ]:
plot_slc_var(df_hnl_prim_shw2, df_nu_prim_shw2, df_cos_prim_shw2,
                    true_counts, start_counts, 
                    'slc_pfp_shower_theta', 
                    xmin = 0, xmax = 180, xnbin = 36,
                    xtitle = 'Shower Theta',
                  ifAddLegend = True, addLegend = label
                    )


plt.axvline(x =  TuneCut['TwoShowerThetaCenter'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(TuneCut['TwoShowerThetaCenter'], 1.7e3), xytext=(TuneCut['TwoShowerThetaCenter'] - 15, 1.7e3)
             , arrowprops=dict(arrowstyle= '<|-',color='r', lw=1.5,ls='--'))
plt.ylim(0, 1.8e3)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"two_shower_primary_theta_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

print(TuneCut['TwoShowerThetaCenter'])

In [ ]:
plot_slc_var(df_hnl_prim_shw2_center, df_nu_prim_shw2_center, df_cos_prim_shw2_center,
#plot_slc_var(df_hnl_prim_shw2_center, df_nu_prim_shw2_center, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_theta', 
                    xmin = 0, xmax = 180, xnbin = 36,
                    xtitle = 'Shower Theta',
                  ifAddLegend = True, addLegend = label
                    )


plt.axvline(x =  TuneCut['TwoShowerThetaCenter'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(TuneCut['TwoShowerThetaCenter'], 1.4e3), xytext=(TuneCut['TwoShowerThetaCenter'] - 15, 1.4e3)
             , arrowprops=dict(arrowstyle= '<|-',color='r', lw=1.5,ls='--'))
plt.ylim(0, 1.5e3)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"two_shower_primary_theta_peak_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

print(TuneCut['TwoShowerThetaCenter'])

In [ ]:
plot_slc_var(df_hnl_prim_shw2_edge, df_nu_prim_shw2_edge, df_cos_prim_shw2_edge,
#plot_slc_var(df_hnl_prim_shw2_edge, df_nu_prim_shw2_edge, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_theta', 
                    xmin = 0, xmax = 180, xnbin = 36,
                    xtitle = 'Shower Theta',
                  ifAddLegend = True, addLegend = label
                    )


plt.axvline(x =  TuneCut['TwoShowerThetaEdge'], c = 'r', linestyle = '--', lw = 1.5)

plt.annotate("", xy=(TuneCut['TwoShowerThetaEdge'], 3.4e2), xytext=(TuneCut['TwoShowerThetaEdge'] - 15, 3.4e2)
             , arrowprops=dict(arrowstyle= '<|-',color='r', lw=1.5,ls='--'))
plt.ylim(0, 3.5e2)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"two_shower_primary_theta_edge_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

print(TuneCut['TwoShowerThetaEdge'])

In [ ]:
if ifTune:
    vary_Theta(df_hnl_prim_shw2_center, df_hnl_prim_shw2_center, df_hnl_prim_shw2_center, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/two_shower_prim_theta_center_tuning.png", dpi =200)
        
    vary_Theta(df_hnl_prim_shw2_edge, df_hnl_prim_shw2_edge, df_hnl_prim_shw2_edge, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/two_shower_prim_theta_edge_tuning.png", dpi =200)

<h3> 2+ showers: cut on primary + less harsh cut </h3>

In [ ]:
if ifCut:
    angle_center = TuneCut['TwoShowerThetaCenter']
    print(angle_center)
    df_hnl_prim_shw2_center = cutThetaAngle(df_hnl_prim_shw2_center, thetaAngle = angle_center) 
    df_nu_prim_shw2_center = cutThetaAngle(df_nu_prim_shw2_center, thetaAngle = angle_center) 
    df_cos_prim_shw2_center = cutThetaAngle(df_cos_prim_shw2_center, thetaAngle = angle_center) 
    
    angle_edge = TuneCut['TwoShowerThetaEdge']
    print(angle_edge)
    df_hnl_prim_shw2_edge = cutThetaAngle(df_hnl_prim_shw2_edge, thetaAngle = angle_edge) 
    df_nu_prim_shw2_edge = cutThetaAngle(df_nu_prim_shw2_edge, thetaAngle = angle_edge) 
    df_cos_prim_shw2_edge = cutThetaAngle(df_cos_prim_shw2_edge, thetaAngle = angle_edge) 
    
df_hnl_prim_shw2 = pd.concat([df_hnl_prim_shw2_center, df_hnl_prim_shw2_edge])
df_nu_prim_shw2 = pd.concat([df_nu_prim_shw2_center, df_nu_prim_shw2_edge])
df_cos_prim_shw2 = pd.concat([df_cos_prim_shw2_center, df_cos_prim_shw2_edge])

<h3> Merge one and two showers back together </h3>

In [ ]:
df_hnl_shw2 = merge_df_prim(df_hnl_shw2, df_hnl_prim_shw2)
df_nu_shw2 = merge_df_prim(df_nu_shw2, df_nu_prim_shw2)
df_cos_shw2 = merge_df_prim(df_cos_shw2, df_cos_prim_shw2)

df_hnl = apply_shower_cut(df_hnl, df_hnl_shw1, df_hnl_shw2)
df_nu = apply_shower_cut(df_nu, df_nu_shw1, df_nu_shw2)
df_cos = apply_shower_cut(df_cos, df_cos_shw1, df_cos_shw2)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Shower Theta Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Make Beam Bucket - Post Shower Theta</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',
                  ifAddLegend = True, addLegend = label
                    )

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postshowertheta.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>2 Shower Case: Pi0 Mass </h1>

<h3> Split into 1 shower and multi shower case again! </h3>

In [ ]:
df_cos['slc_pfp_shower_theta']

In [ ]:
df_hnl_shw1, df_hnl_shw2 = split_my_df(df_hnl)

df_nu_shw1, df_nu_shw2 = split_my_df(df_nu)

#df_cos_shw1, df_cos_shw2 = split_my_df(df_cos)

In [ ]:
df_hnl_shw2['pi0_mass'] = df_hnl_shw2.apply(lambda row: pi0mass(row), axis = 1)

df_nu_shw2['pi0_mass'] = df_nu_shw2.apply(lambda row: pi0mass(row), axis = 1)

df_cos['pi0_mass']  = ""
#df_cos_shw2['pi0_mass'] = df_cos_shw2.apply(lambda row: pi0mass(row), axis = 1)


In [ ]:
df_hnl_prim_shw2 = get_primary_shw_df(df_hnl_shw2)

df_nu_prim_shw2 = get_primary_shw_df(df_nu_shw2)

#df_cos_prim_shw2 = get_primary_shw_df(df_cos_shw2)
df_cos_prim_shw2 = get_primary_shw_df(df_cos)

In [ ]:
when_hnl_shw2_center = (df_hnl_prim_shw2['mod_t'] >= lb) &  (df_hnl_prim_shw2['mod_t'] <= ub)
when_nu_shw2_center = (df_nu_prim_shw2['mod_t'] >= lb) &  (df_nu_prim_shw2['mod_t'] <= ub)
when_cos_shw2_center = (df_cos_prim_shw2['mod_t'] >= lb) &  (df_cos_prim_shw2['mod_t'] <= ub)

when_hnl_shw2_edge = (df_hnl_prim_shw2['mod_t'] < lb) | (df_hnl_prim_shw2['mod_t'] > ub)
when_nu_shw2_edge = (df_nu_prim_shw2['mod_t'] < lb) | (df_nu_prim_shw2['mod_t'] > ub)
when_cos_shw2_edge = (df_cos_prim_shw2['mod_t'] < lb) | (df_cos_prim_shw2['mod_t'] > ub)

df_hnl_prim_shw2_center = df_hnl_prim_shw2[when_hnl_shw2_center]
df_nu_prim_shw2_center = df_nu_prim_shw2[when_nu_shw2_center]
df_cos_prim_shw2_center = df_cos_prim_shw2[when_cos_shw2_center]


df_hnl_prim_shw2_edge = df_hnl_prim_shw2[when_hnl_shw2_edge]
df_nu_prim_shw2_edge = df_nu_prim_shw2[when_nu_shw2_edge]
df_cos_prim_shw2_edge = df_cos_prim_shw2[when_cos_shw2_edge]

In [ ]:
mass_lb = 0
mass_ub = 300

In [ ]:
plot_slc_var(df_hnl_prim_shw2, df_nu_prim_shw2, df_cos,
#plot_slc_var(df_hnl_prim_shw2, df_nu_prim_shw2, df_cos_prim_shw2,
                    true_counts, start_counts, 
                    'pi0_mass', 
                    xmin = 0, xmax = 600, xnbin = 30,
                    xtitle = 'Pi 0 Mass [MeV]',
                  ifAddLegend = True, addLegend = label
                    )
# only one line may be specified; full height
plt.axvline(x = 135, color = 'r', linewidth = 2.5, ls = '-')
plt.axvline(x = mass_lb, color = 'r', linewidth = 1.5, ls = '--')
plt.axvline(x = mass_ub, color = 'r', linewidth = 1.5, ls = '--')

plt.annotate("", xy=(mass_ub, 1.7e3), xytext=(mass_ub - 50, 1.7e3)
             , arrowprops=dict(arrowstyle= '<|-',color='r', lw=1.5,ls='--'))
plt.ylim(0, 1.8e3)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"pizero_mass_precut.png", dpi = 200)
#-----------------------------------#

plt.show()

In [ ]:
plot_slc_var(df_hnl_prim_shw2_center, df_nu_prim_shw2_center, df_cos,
#plot_slc_var(df_hnl_prim_shw2, df_nu_prim_shw2, df_cos_prim_shw2,
                    true_counts, start_counts, 
                    'pi0_mass', 
                    xmin = 0, xmax = 600, xnbin = 30,
                    xtitle = 'Pi 0 Mass [MeV]',
                  ifAddLegend = True, addLegend = label
                    )
# only one line may be specified; full height
plt.axvline(x = 135, color = 'r', linewidth = 2.5, ls = '-')
plt.axvline(x = mass_lb, color = 'r', linewidth = 1.5, ls = '--')
plt.axvline(x = mass_ub, color = 'r', linewidth = 1.5, ls = '--')
plt.annotate("", xy=(mass_ub, 1.5e3), xytext=(mass_ub - 50, 1.5e3)
             , arrowprops=dict(arrowstyle= '<|-',color='r', lw=1.5,ls='--'))
plt.ylim(0, 1.6e3)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"pizero_mass_peak_precut.png", dpi = 200)
#-----------------------------------#

plt.show()

In [ ]:
plot_slc_var(df_hnl_prim_shw2_edge, df_nu_prim_shw2_edge, df_cos,
#plot_slc_var(df_hnl_prim_shw2, df_nu_prim_shw2, df_cos_prim_shw2,
                    true_counts, start_counts, 
                    'pi0_mass', 
                    xmin = 0, xmax = 600, xnbin = 30,
                    xtitle = 'Pi 0 Mass [MeV]',
                  ifAddLegend = True, addLegend = label
                    )
# only one line may be specified; full height
plt.axvline(x = 135, color = 'r', linewidth = 2.5, ls = '-')
plt.axvline(x = mass_lb, color = 'r', linewidth = 1.5, ls = '--')
plt.axvline(x = mass_ub, color = 'r', linewidth = 1.5, ls = '--')
plt.annotate("", xy=(mass_ub, 2.4e2), xytext=(mass_ub - 50, 2.4e2)
             , arrowprops=dict(arrowstyle= '<|-',color='r', lw=1.5,ls='--'))
plt.ylim(0, 2.5e2)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"pizero_mass_edge_precut.png", dpi = 200)
#-----------------------------------#

plt.show()

<h3>Apply Cut</h3>

In [ ]:
if ifCut:

    df_hnl_prim_shw2_center =  df_hnl_prim_shw2_center[(df_hnl_prim_shw2_center['pi0_mass'] >= mass_lb) & (df_hnl_prim_shw2_center['pi0_mass'] <= mass_ub)]
    df_nu_prim_shw2_center = df_nu_prim_shw2_center[(df_nu_prim_shw2_center['pi0_mass'] >= mass_lb) & (df_nu_prim_shw2_center['pi0_mass'] <= mass_ub)]
    df_cos_prim_shw2_center = df_cos_prim_shw2_center[(df_cos_prim_shw2_center['pi0_mass'] >= mass_lb) & (df_cos_prim_shw2_center['pi0_mass'] <= mass_ub)] 
    
    df_hnl_prim_shw2_edge =  df_hnl_prim_shw2_edge[(df_hnl_prim_shw2_edge['pi0_mass'] >= mass_lb) & (df_hnl_prim_shw2_edge['pi0_mass'] <= mass_ub)]
    df_nu_prim_shw2_edge = df_nu_prim_shw2_edge[(df_nu_prim_shw2_edge['pi0_mass'] >= mass_lb) & (df_nu_prim_shw2_edge['pi0_mass'] <= mass_ub)]
    df_cos_prim_shw2_edge = df_cos_prim_shw2_edge[(df_cos_prim_shw2_edge['pi0_mass'] >= mass_lb) & (df_cos_prim_shw2_edge['pi0_mass'] <= mass_ub)] 
    
    
df_hnl_prim_shw2 = pd.concat([df_hnl_prim_shw2_center, df_hnl_prim_shw2_edge])
df_nu_prim_shw2 = pd.concat([df_nu_prim_shw2_center, df_nu_prim_shw2_edge])
df_cos_prim_shw2 = pd.concat([df_cos_prim_shw2_center, df_cos_prim_shw2_edge])

In [ ]:
df_hnl_shw2 = merge_df_prim(df_hnl_shw2, df_hnl_prim_shw2)

df_nu_shw2 = merge_df_prim(df_nu_shw2, df_nu_prim_shw2)

df_cos_shw2 = merge_df_prim(df_cos_shw2, df_cos_prim_shw2)

In [ ]:
df_hnl = apply_shower_cut(df_hnl, df_hnl_shw1, df_hnl_shw2)

df_nu = apply_shower_cut(df_nu, df_nu_shw1, df_nu_shw2)

df_cos = apply_shower_cut(df_cos, df_cos_shw1, df_cos_shw2)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Pion Mass Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1>Beam Bucket - Post Pion Mass Cut</h1>

In [ ]:
hist, bins = plot_slc_var(
                    df_hnl
                    , df_nu
                    , df_cos
                    , true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',
                  ifAddLegend = True, addLegend = label
                    )

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postpionmass.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Beam Timing Cut</h1>

In [ ]:
when_hnl_edge = (df_hnl['mod_t'] < lb) | (df_hnl['mod_t'] > ub)
when_nu_edge = (df_nu['mod_t'] < lb) | (df_nu['mod_t'] > ub)
when_cos_edge = (df_cos['mod_t'] < lb) | (df_cos['mod_t'] > ub)

df_hnl_edge = df_hnl[when_hnl_edge]
df_nu_edge = df_nu[when_nu_edge]
df_cos_edge = df_cos[when_cos_edge]

<h3> Save Purity/Eff </h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl_edge, df_nu_edge, df_cos_edge, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Timing Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1>Beam Bucket Final Distribution</h1>

In [ ]:
df_hnl = get_primary_shw_df(df_hnl)

df_nu = get_primary_shw_df(df_nu)

df_cos = get_primary_shw_df(df_cos)

In [ ]:
when_hnl_center = (df_hnl['mod_t'] >= lb) &  (df_hnl['mod_t'] <= ub)
when_nu_center = (df_nu['mod_t'] >= lb) &  (df_nu['mod_t'] <= ub)
when_cos_center = (df_cos['mod_t'] >= lb) &  (df_cos['mod_t'] <= ub)

when_hnl_edge = (df_hnl['mod_t'] < lb) | (df_hnl['mod_t'] > ub)
when_nu_edge = (df_nu['mod_t'] < lb) | (df_nu['mod_t'] > ub)
when_cos_edge = (df_cos['mod_t'] < lb) | (df_cos['mod_t'] > ub)

df_hnl_center = df_hnl[when_hnl_center]
df_nu_center = df_nu[when_nu_center]
df_cos_center = df_cos[when_cos_center]

df_hnl_edge = df_hnl[when_hnl_edge]
df_nu_edge = df_nu[when_nu_edge]
df_cos_edge = df_cos[when_cos_edge]

In [ ]:
hist, bins = plot_slc_var(
                    df_hnl
                    , df_nu
                    , df_cos
                    , true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',
                  ifAddLegend = True, addLegend = label
                    )

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_final.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
hist, bins = plot_slc_var(
                    df_hnl_center
                    , df_nu_center
                    , df_cos_center
                    , true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',
                  ifAddLegend = True, addLegend = label
                    )

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_peak_final.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
hist, bins = plot_slc_var(
                    df_hnl_edge#[df_hnl_edge['slc_pfp_shower_energy'] >= 100]
                    , df_nu_edge#[df_nu_edge['slc_pfp_shower_energy'] >= 100]
                    , df_cos_edge#[df_cos_edge['slc_pfp_shower_energy'] >= 100]
                    , true_counts, start_counts, 
                    'mod_t', 
                    xmin = 0, xmax = 30, xnbin = 30,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',
                  ifAddLegend = True, addLegend = label
                    )
#plt.ylim(0,450)
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_edge_final.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Plot Purity/Efficiency </h1>

In [ ]:
print(eff_arr)
print(purity_arr)
print(cut_arr)

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(10, 6), sharex = True)
ax2 = ax1.twinx()

t = np.arange(0, len(cut_arr) , 1)

effcol = col_dict['MintGreen']
pcol = col_dict['Flamingo']

ax1.plot(t, eff_arr, c = effcol, marker = 'o', markersize = 5, ls='--')
ax2.plot(t, purity_arr, c = pcol, marker = 'o', markersize = 5, ls='--')

ax1.set_ylim(-5,105)
ax2.set_ylim(-5,105)

ax1.grid(color='gainsboro', which='both', axis='both', linestyle='--')

ax1.set_ylabel("Selection Effciency [%]", c = effcol, fontsize = 16)
ax2.set_ylabel("Purity [%]", c = pcol, fontsize = 16)

plt.xticks(range(0, len(t)))
ax1.set_xticklabels(cut_arr, rotation = 45, ha="right", fontsize = 16)

ax1.tick_params(bottom = True, top = True, left = True, right = False)
ax2.tick_params(bottom = True, top = True, left = False, right = True)

ax1.tick_params(axis = 'x', labelsize = 16, direction = 'in')
ax1.tick_params(axis = 'y', labelsize = 16, direction = 'in')
ax2.tick_params(axis = 'y', labelsize = 16, direction = 'in')

ax1.set_title(label, y=1.0, pad=-55, fontsize = 16)
fig.tight_layout()

#-----------------------------------#
if ifSave:
    plt.savefig(savePath+"peff.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Save Only Primary Showers </h1>

In [ ]:
df_hnl = get_primary_shw_df(df_hnl)

df_nu = get_primary_shw_df(df_nu)

df_cos = get_primary_shw_df(df_cos)
#df_cos_prim_shw2 = get_primary_shw_df(df_cos)

In [ ]:
df_hnl[df_hnl['slc_true_event_type'] == 9]

In [ ]:
df_nu[df_nu['slc_true_event_type'] == 9]

In [ ]:
df_cos

<h1> Save Cut Dataframe to Pickle</h1>

In [ ]:
df_hnl.to_pickle("../../pkl_files/v3_April2024/df_postselect_truly_final_loosy_m"+str(m)+"_v3_hnl.pkl", protocol = 5)